# ODS to output
process and output




| Modified by |  Last modification | Task Number | Description |
| ----------- | ----------- | ----------- |----------- |
| Luca Barcella | 2025-04-08 |000000|Create notebook|


In [16]:
ENV = None
PROJECT_NAME = "ods"
process_date = None

#Metadata store
admin_schema_name = "admin"
data_lineage_table_name = "data_lineage"
admin_table_name_to_output = "ods_metadata_to_output_v1"

## Utility Functions

In [17]:
%run "Utils/data_processing_utilis_v2"

In [18]:
if not process_date:
    process_date = date.today().strftime('%Y-%m-%d')
    print(f"Process date has not been declared, new process date is: {process_date}")

In [ ]:
def cast_dataframe_to_schema(source_df, target_df):
    """
    Cast all columns from source_df to match the exact data types in target_df.
    This ensures schema compatibility and prevents type mismatch errors during append operations.
    
    Args:
        source_df: Source DataFrame whose columns will be cast
        target_df: Target DataFrame whose schema will be used as reference
    
    Returns:
        DataFrame with columns from source_df cast to target_df data types
    """
    # Find common columns between both tables
    common_columns = sorted(list(set(source_df.columns) & set(target_df.columns)))
    
    # Create a mapping of column names to their target data types
    target_schema = {field.name: field.dataType for field in target_df.schema.fields}
    
    # Cast each column to the target type
    result_df = source_df.select([
        F.col(col_name).cast(target_schema[col_name]) 
        for col_name in common_columns
    ])
    
    return result_df

# <mark>Code</mark>

## Source system ID

In [20]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema":"output", "target_table":"ignore", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"table":"dbo.GlobalOptionsetMetadata", "view":"GlobalOptionsetMetadata", "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

# Get Core source system ID  - 1
core_source_system_id = spark.sql("""
    SELECT      DISTINCT Option 
    FROM        GlobalOptionsetMetadata
    WHERE       OptionSetName = 'lk_sourcesystemunit'
    AND         lower(LocalizedLabel) = lower('Mars Core systems (ATLAS/MDG/Veritas)')
    LIMIT 1
""").collect()[0][0]
print(f"core_source_system_id: {core_source_system_id}")

# Get UK Kind source system ID - 2
uk_kind_source_system_id = spark.sql("""
    SELECT      DISTINCT Option 
    FROM        GlobalOptionsetMetadata
    WHERE       OptionSetName = 'lk_sourcesystemunit'
    AND         lower(LocalizedLabel) = lower('UK Kind')
    LIMIT 1
""").collect()[0][0]
print(f"uk_kind_source_system_id: {uk_kind_source_system_id}")

# Get Fusion source system ID -3
fusion_source_system_id = spark.sql("""
    SELECT      DISTINCT Option 
    FROM        GlobalOptionsetMetadata
    WHERE       OptionSetName = 'lk_sourcesystemunit'
    AND         lower(LocalizedLabel) = lower('Fusion')
    LIMIT 1
""").collect()[0][0]
print(f"fusion_source_system_id: {fusion_source_system_id}")


## dim_plant

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "dim_plant", "format":"delta","opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
        {"view_name":"dimentitieshierarchy", "path":"environments/s4o/prod/s4o_process/dimentitieshierarchy", "file_format":"parquet", "container":"synapse" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service", "csv_options":{ "inferSchema": "true", "header": "true"}},
        {"view_name":"plant_site_code_xref", "path":"SKUPRINT/plant_site_code_xref.csv", "file_format":"csv", "container":"files" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service", "csv_options":{ "inferSchema": "true", "header": "true"}},
        {"view_name":"0plant_attr_aep",  "path":"ATLAS/AEP/MASTER_DATA/0PLANT_ATTR/DATA", "container":"output" ,"linked_service_name":"MARS_ANALYTICS_ADLS_LS", "opertation_type":"Linked Service"},
        {"view_name":"0plant_attr_aap",  "path":"ATLAS/AAP/MASTER_DATA/0PLANT_ATTR/DATA", "container":"output" ,"linked_service_name":"MARS_ANALYTICS_ADLS_LS", "opertation_type":"Linked Service"},
        {"view_name":"0plant_attr_app",  "path":"ATLAS/APP/MASTER_DATA/0PLANT_ATTR/DATA", "container":"output" ,"linked_service_name":"MARS_ANALYTICS_ADLS_LS", "opertation_type":"Linked Service"},
		{"table":"dbo.v_odsrules",      "view":"v_odsrules",    "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

df_atlas = spark.sql("""
    WITH Enablon AS (
        SELECT  DISTINCT
                Country                                         AS enablon_country,
                Organization                                    AS enablon_organization,
                Segment                                         AS enablon_segment,
                Division                                        AS enablon_division,
                Business_Unit                                   AS enablon_business_unit,
                Site                                            AS enablon_site
        FROM    dimentitieshierarchy
    ),
    Xref AS (
        SELECT  plant,
                site_code
        FROM    plant_site_code_xref
    ),
    0plant_attr_union AS (
        SELECT  WERKS_PK                                        AS plant_code,
                NAME1                                           AS plant_name,
                VKORG                                           AS sales_organisation,
                LAND1                                           AS country_code,
                BUKRS                                           AS company_code,
                replace(ltrim(replace(KUNNR,'0',' ')),' ','0')  AS customer_number,
                replace(ltrim(replace(LIFNR,'0',' ')),' ','0')  AS vendor_number,
                _MARS_SITECODE                                  AS site_code_atlas,
                LOAD_DATE
        FROM    0plant_attr_aep

        UNION 

        SELECT  WERKS_PK                                        AS plant_code,
                NAME1                                           AS plant_name,
                VKORG                                           AS sales_organisation,
                LAND1                                           AS country_code,
                BUKRS                                           AS company_code,
                replace(ltrim(replace(KUNNR,'0',' ')),' ','0')  AS customer_number,
                replace(ltrim(replace(LIFNR,'0',' ')),' ','0')  AS vendor_number,
                _MARS_SITECODE                                  AS site_code_atlas,
                LOAD_DATE
        FROM    0plant_attr_aap

        UNION

        SELECT  WERKS_PK                                        AS plant_code,
                NAME1                                           AS plant_name,
                VKORG                                           AS sales_organisation,
                LAND1                                           AS country_code,
                BUKRS                                           AS company_code,
                replace(ltrim(replace(KUNNR,'0',' ')),' ','0')  AS customer_number,
                replace(ltrim(replace(LIFNR,'0',' ')),' ','0')  AS vendor_number,
                _MARS_SITECODE                                  AS site_code_atlas,
                LOAD_DATE
        FROM    0plant_attr_app
    ),
    Atlas AS (
        SELECT  DISTINCT 
                *
        FROM (
            SELECT  plant_code,
                    LAST_VALUE(plant_name, TRUE) OVER (PARTITION BY plant_code ORDER BY LOAD_DATE ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS plant_name,
                    LAST_VALUE(sales_organisation, TRUE) OVER (PARTITION BY plant_code ORDER BY LOAD_DATE ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS sales_organisation,
                    LAST_VALUE(country_code, TRUE) OVER (PARTITION BY plant_code ORDER BY LOAD_DATE ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS country_code,
                    LAST_VALUE(company_code, TRUE) OVER (PARTITION BY plant_code ORDER BY LOAD_DATE ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS company_code,
                    LAST_VALUE(customer_number, TRUE) OVER (PARTITION BY plant_code ORDER BY LOAD_DATE ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS customer_number,
                    LAST_VALUE(vendor_number, TRUE) OVER (PARTITION BY plant_code ORDER BY LOAD_DATE ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS vendor_number
            FROM    0plant_attr_union
        )
    )

    SELECT      a.*,
                x.*,
                e.*,
                '1'                             AS SourceSystemUnit,
                a.plant_code                    AS src_agnostic_unique_ID    
    FROM        Atlas a 
    LEFT JOIN   Xref x 
        ON      a.plant_code = x.plant
    LEFT JOIN   Enablon e 
        ON      x.site_code = e.enablon_site

""")

df_system_agnostic = spark.sql("""
    SELECT  value2                              AS plant_code,
            value3                              AS plant_name,
            value4                              AS country_code,
            CONCAT(value1, ':', value5)         AS company_code,
            CONCAT(value1, ':', value6)         AS sales_organisation,
            value1                              AS SourceSystemUnit,
            CONCAT(value1, ':', value2)         AS src_agnostic_unique_ID
    FROM    v_odsrules
    WHERE   function = 'SYSTEM_AGNOSTIC_SETUP'
        AND criteria = 'PLANT'
""")

df_joined = df_atlas.unionByName(df_system_agnostic, allowMissingColumns=True).withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df_joined)

## dim_material

### dim_material <u>CORE</u> 
Atlas/Veritas

In [23]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema":"output", "target_table":"dim_material", "format":"delta","opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
        {"schema":"normalized", "table":"purchase_order_line_item",             "view":"purchase_order_line_item",          "project_name":"atlas",     "params":{"ignore_env": true}},
        {"schema":"normalized", "table":"process_order",                        "view":"process_order",                     "project_name":"atlas",     "params":{"ignore_env": true}},
        {"schema":"normalized", "table":"mdg_classification_zzglobal01",        "view":"mdg_classification_zzglobal01_decoded"},
        {"schema":"normalized", "table":"mdg_classification_zzpack01",          "view":"mdg_classification_zzpack01_decoded"},
        {"schema":"normalized", "table":"mdg_classification_zzraw01",           "view":"mdg_classification_zzraw01_decoded"},
        {"schema":"normalized", "table":"mdg_classification_zzudc",             "view":"mdg_classification_zzudc_decoded"},
        {"schema":"output",     "table":"dim_plant",                            "view":"dim_plant"},
        {"schema":"raw",        "table":"mdg_regional_codes",                   "view":"mdg_regional_codes"},
        {"schema":"raw",        "table":"mdg_material_group",                   "view":"mdg_material_group"},
        {"schema":"raw",        "table":"atlas_sievo_categories",               "view":"atlas_sievo_categories"},

        {"schema":"raw",        "table":"mdg_finished_products_fert",           "view":"mdg_finished_products_fert"},
        {"schema":"raw",        "table":"mdg_packaging_and_pack_verp",          "view":"mdg_packaging_and_pack_verp"},
        {"schema":"raw",        "table":"mdg_hierarchy_node_materials_zhie",    "view":"mdg_hierarchy_node_materials_zhie"},
        {"schema":"raw",        "table":"mdg_raw_material_roh",                 "view":"mdg_raw_material_roh"},
        {"schema":"raw",        "table":"mdg_food_contact_zqfc",                "view":"mdg_food_contact_zqfc"},
        {"schema":"raw",        "table":"mdg_unval_material_rent_pal_unbw",     "view":"mdg_unval_material_rent_pal_unbw"},
        {"schema":"raw",        "table":"mdg_representative_item_zrep",         "view":"mdg_representative_item_zrep"},

        {"schema":"raw",        "table":"helios_item_taxonomy",                 "view":"helios_item_taxonomy"},
        {"schema":"raw",        "table":"helios_it_copack_group",               "view":"helios_it_copack_group"},
        {"schema":"raw",        "table":"helios_it_pack_size",                  "view":"helios_it_pack_size"},
        {"schema":"raw",        "table":"helios_it_tech",                       "view":"helios_it_tech"},
        {"schema":"raw",        "table":"helios_it_intext_mfg_group",           "view":"helios_it_intext_mfg_group"},
        {"schema":"raw",        "table":"helios_it_ec_group",                   "view":"helios_it_ec_group"},
        {"schema":"raw",        "table":"helios_it_financial_product_segment",  "view":"helios_it_financial_product_segment"},
        {                       "table":"dbo.v_odsrules",                       "view":"v_odsrules",    "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

print(f"core_source_system_id: {core_source_system_id}")

# ======================================================================================================================================================
# =====================                        1. Combine material data by material type from RAW                                  =====================
# ======================================================================================================================================================

from functools import reduce
from pyspark.sql import DataFrame

fields = [
    "Material", "Description", "Representative_Item", "Material_Type", "Created_By", "Created_On", "Gross_Weight", "Net_Weight", "Weight_Unit", 
        "X_Plant_Status", "Length", "Width", "Height", "EAN", "EAN_Category", "Unit_of_Dimension", "Material_Group", "Base_Unit_of_Measure", "Document",
        "Total_shelf_life", "Min_Rem_Shelf_Life", "Merchandising_Unit", "Retail_Sales_Unit", "Traded_Unit", "Intermediate_Product_Component", "Semi_Finished_Product"
]

# list of source tables
tables = [
    "mdg_finished_products_fert",                                   # FERT
    "mdg_packaging_and_pack_verp",                                  # VERP
    "mdg_food_contact_zqfc",                                        # ZGFC
    "mdg_hierarchy_node_materials_zhie",                            # ZHIE
    "mdg_raw_material_roh",                                         # ROH
    "mdg_unval_material_rent_pal_unbw",                             # UNBW
    "mdg_representative_item_zrep"                                  # ZREP
]

dfs = [spark.table(t).select(fields) for t in tables]               # build list of DataFrames
df = reduce(DataFrame.unionByName, dfs)                             # union them all
df.createOrReplaceTempView("df_material")                           # create view


# ======================================================================================================================================================
# =====================                        2. Establish list of Sourcing Plants by material                                    =====================
# ======================================================================================================================================================

df = spark.sql(f"""
    WITH Orders AS (
        SELECT  Material_Number,
                Base_Unit_of_Measure, 
                Plant, 
                CAST(Quantity_of_goods_received as FLOAT) as Quantity, 
                "Order" as OrderType
        FROM    process_order 
        UNION ALL
        SELECT  Material_Number,
                Base_Unit_of_Measure, 
                Plant , 
                CAST(Purchase_Order_Quantity as FLOAT)  as Quantity, 
                "Purchase" as OrderType
        FROM    purchase_order_line_item 
    )

    SELECT      o.Material_Number, 
                o.Plant, 
                o.Quantity, 
                o.OrderType, 
                d.plant_name, 
                o.Base_Unit_of_Measure
    FROM        Orders o 
    LEFT JOIN   dim_plant d 
        ON      o.Plant = d.plant_code
""")

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import sum as spark_sum, round as spark_round, to_date, concat_ws, collect_list, sort_array

df = df.withColumn("Quantity_numeric", col("Quantity").cast("double"))

# Data aggregation
aggregated_df = df.groupBy("Material_Number", "Plant", "plant_name", "Base_Unit_of_Measure").agg(spark_sum("Quantity_numeric").alias("Total_Quantity"))

# Calculate total quantity for each Material_Number
total_quantity_df = df.groupBy("Material_Number").agg(spark_sum("Quantity_numeric").alias("Total_Quantity_All_Plants"))

# Join data
joined_df = aggregated_df.join(total_quantity_df, ["Material_Number"])

# Calculate percentage share
result_df = joined_df.withColumn(
    "Percentage",
    spark_round((col("Total_Quantity") / col("Total_Quantity_All_Plants")) * 100, 2)
)

# Creating a summary column
result_df = result_df.withColumn(
    "Plant_Details",
    concat_ws(": ",  concat_ws(" - ", col("Plant") , col("plant_name"), col("Base_Unit_of_Measure") ), spark_round(col("Total_Quantity")).cast(IntegerType()), 
              concat_ws("", lit(" ("), col("Percentage"), lit("%)")))
)

# Creating a summary column
df = result_df.groupBy("Material_Number").agg(
    concat_ws("; ", sort_array(collect_list(col("Plant_Details")))).alias("Sourcing_Plant")
)

# Aggregation into the final formatSave result to target table
df.createOrReplaceTempView("df_sourcing_plant_per_material")


# ======================================================================================================================================================
# =====================                                     3. Establish Manufacturing Type                                        =====================
# ======================================================================================================================================================

spark.sql("""
    SELECT      Material_Number,
                MAX(IsComan) as IsComan,
                MAX(IsCopack) as IsCopack,
                MAX(IsInhouse) as IsInhouse
    FROM (
        SELECT      d.Material AS Material_Number, 
                    CAST(1 AS BOOLEAN) as IsComan,
                    CAST(0 AS BOOLEAN) as IsCopack,
                    CAST(0 AS BOOLEAN) as IsInhouse
        FROM        df_material d
        SEMI JOIN   purchase_order_line_item p
            ON      p.Material_Number = d.Material
            AND     p.Purchasing_Document_Type = 'NB'
            AND     p.Item_category_in_purchasing_document IN (0)
        WHERE       d.Material_Type = 'FERT'

        UNION

        SELECT      d.Material AS Material_Number,
                    CAST(0 AS BOOLEAN) as IsComan,
                    CAST(1 AS BOOLEAN) as IsCopack,
                    CAST(0 AS BOOLEAN) as IsInhouse
        FROM        df_material d
        SEMI JOIN   purchase_order_line_item p
            ON      p.Material_Number = d.Material
            AND     p.Purchasing_Document_Type = 'NB'
            AND     p.Item_category_in_purchasing_document IN (3)
        WHERE       d.Material_Type = 'FERT'

        UNION

        SELECT  Material_Number,
                CAST(0 AS BOOLEAN) as IsComan,
                CAST(0 AS BOOLEAN) as IsCopack,
                CAST(1 AS BOOLEAN) as IsInhouse
        FROM    process_order
    )
    GROUP BY    Material_Number
""").createOrReplaceTempView("df_manufacturing_type_per_material")


# ======================================================================================================================================================
# =====================                                 4. Prepare HELIOS taxonomy input                                           =====================
# ======================================================================================================================================================

spark.sql("""
    SELECT      DISTINCT
                t.Material,
                t.IT_CoPack_Group_ID,
                cg.Description Copack_Group,
                t.IT_Pack_Size_ID,
                ps.Description Pack_Size,
                t.IT_Tech_ID,
                it.Description Tech,
                t.IT_IntExt_Mfg_Group_ID,
                int.Description IntExt_Mfg_Group,
                t.IT_EC_Group_ID,
                eg.Description EC_Group,
                t.IT_Financial_Product_Segment_ID,
                fps.Description AS Financial_Product_Segment
    FROM        helios_item_taxonomy t
    LEFT JOIN   helios_it_copack_group cg
        ON      t.IT_CoPack_Group_ID = cg.IT_CoPack_Group_ID
    LEFT JOIN   helios_it_pack_size ps 
        ON      t.IT_Pack_Size_ID = ps.IT_Pack_Size_ID
    LEFT JOIN   helios_it_tech it 
        ON      t.IT_Tech_ID = it.IT_Tech_ID
    LEFT JOIN   helios_it_intext_mfg_group int
        ON      t.IT_IntExt_Mfg_Group_ID = int.IT_IntExt_Mfg_Group_ID
    LEFT JOIN   helios_it_ec_group eg 
        ON      t.IT_EC_Group_ID = eg.IT_EC_Group_ID
    LEFT JOIN   helios_it_financial_product_segment fps
        ON      t.IT_Financial_Product_Segment_ID = fps.IT_Financial_Product_Segment_ID
""").createOrReplaceTempView("df_helios_material_taxonomy")


# ======================================================================================================================================================
# =====================                                     5. Prepare final output                                                =====================
# ======================================================================================================================================================

from pyspark.sql import functions as F

spark.sql("""
    SELECT      m.Material,
                m.Description,
                m.Representative_Item,
                regio_codes.Veritas_Spec_Number,
                m.Material_Type,
                mft.IsComan,
                mft.IsCopack,
                mft.IsInhouse,
                sp.Sourcing_Plant,
                mg.Material_Group_Long_Description,
                m.Created_By,
                m.Created_On,
                m.Gross_Weight,
                CASE    WHEN m.Weight_Unit='KG' THEN m.Net_Weight * 1000.00
                        WHEN m.Weight_Unit='G'  THEN m.Net_Weight
                        WHEN m.Weight_Unit='LB' THEN m.Net_Weight * 453.59237
                        WHEN m.Weight_Unit='OZ' THEN m.Net_Weight * 28.3495231
                        WHEN m.Weight_Unit='MG' THEN m.Net_Weight / 1000.00
                        WHEN m.Weight_Unit='TO' THEN m.Net_Weight * 1000000.00
                        ELSE m.Weight_Unit END                                          AS Net_Weight_Grams,
                m.Net_Weight,
                m.Weight_Unit,
                m.X_Plant_Status,
                CASE    WHEN m.X_Plant_Status=10 THEN 'Active'
                        WHEN m.X_Plant_Status=40 THEN 'Development' 
                        WHEN m.X_Plant_Status=50 THEN 'Creation'
                        WHEN m.X_Plant_Status=90 THEN 'Retired' END                     AS X_Plant_Status_Desc,
                CASE    WHEN m.Weight_Unit IS NOT NULL THEN 1*1000000 / (
                            CASE    WHEN m.Weight_Unit='KG' THEN m.Net_Weight * 1000.00
                                    WHEN m.Weight_Unit='G'  THEN m.Net_Weight
                                    WHEN m.Weight_Unit='LB' THEN m.Net_Weight * 453.59237
                                    WHEN m.Weight_Unit='OZ' THEN m.Net_Weight * 28.3495231
                                    WHEN m.Weight_Unit='MG' THEN m.Net_Weight / 1000.00
                                    WHEN m.Weight_Unit='TO' THEN m.Net_Weight * 1000000.00
                                    ELSE m.Weight_Unit END)
                        ELSE m.Weight_Unit END                                          AS Quantity_of_1,
                m.Length,
                m.Width,
                m.Height,
                m.EAN,
                m.EAN_Category,
                m.Unit_of_Dimension,
                m.Material_Group,
                m.Base_Unit_of_Measure,
                m.Document,
                m.Total_shelf_life,
                m.Min_Rem_Shelf_Life,
                m.Merchandising_Unit,
                m.Retail_Sales_Unit,
                m.Traded_Unit,
                m.Intermediate_Product_Component,
                m.Semi_Finished_Product,
                zzglobal.Animal_Life_Stage,
                zzglobal.Animal_Breed,
                zzraw.Animal_Parts,
                zzglobal.Animal_Size,
                zzglobal.Animal_Species,
                zzglobal.Brand_Essence,
                zzglobal.CoManufact_Packing_Activity,
                zzglobal.CoPacking_Packing_Activity,
                zzglobal.CoPacking_Product_Format,
                zzglobal.CoPacking_Technology,
                zzglobal.Compliant_for_Countries,
                zzglobal.Cuisine,
                zzglobal.Diet_Claims,
                zzglobal.Display_Storage_Condition,
                zzglobal.Lifestyle,
                zzglobal.On_pack_Consumer_Offer,
                zzglobal.On_pack_Consumer_Value,
                zzglobal.On_pack_Trade_Offer,
                zzglobal.Product_Pack_Size_Group,
                zzglobal.Sustainability,
                zzglobal.Traded_Unit_Configuration,
                zzglobal.Traded_Unit_Format,
                COALESCE(zzglobal.Business_Segment,         zzraw.Business_Segment)                                     AS Business_Segment,
                COALESCE(zzglobal.Market_Segment,           zzraw.Market_Segment)                                       AS Market_Segment,
                COALESCE(zzglobal.Brand_Flag,               zzpack.Brand_Flag,              zzraw.Brand_Flag)           AS Brand_Flag,
                COALESCE(zzglobal.Brand_Sub_Flag,           zzpack.Brand_Sub_Flag,          zzraw.Brand_Sub_Flag)       AS Brand_Sub_Flag,
                COALESCE(zzglobal.Supply_Segment,		    zzpack.Product_Pack_Size)                                   AS Supply_Segment,
                COALESCE(zzglobal.Ingredient_Variety,	    zzraw.Ingredient_Variety)                                   AS Ingredient_Variety,
                COALESCE(zzglobal.Functional_Variety,	    zzraw.Functional_variety)                                   AS Functional_Variety,
                COALESCE(zzglobal.Trade_Sector,			    zzraw.Trade_Sector)                                         AS Trade_Sector,
                COALESCE(zzglobal.Marketing_Concept,	    zzraw.Marketing_Concept)                                    AS Marketing_Concept,
                COALESCE(zzglobal.Multipack_Quantity,	    zzraw.Multipack_Quantity)                                   AS Multipack_Quantity,
                COALESCE(zzglobal.Occasion,				    zzraw.Occasion)                                             AS Occasion,
                COALESCE(zzglobal.Product_Category,		    zzraw.Product_Category)                                     AS Product_Category,
                COALESCE(zzglobal.Product_Type,			    zzraw.Product_Type)                                         AS Product_Type,
                COALESCE(zzglobal.Product_Pack_Size,	    zzpack.Product_Pack_Size,       zzraw.Product_Pack_Size)    AS Product_Pack_Size,
                COALESCE(zzglobal.Consumer_Pack_Type,	    zzraw.Consumer_Pack_Type)                                   AS Consumer_Pack_Type,
                COALESCE(zzglobal.Consumer_Pack_Format,	    zzraw.Consumer_Pack_Format)                                 AS Consumer_Pack_Format,
                COALESCE(zzglobal.FPPS_Minor_Pack,		    zzraw.FPPS_Minor_Pack)                                      AS FPPS_Minor_Pack,
                zzpack.US_Value_Chain_Segment,
                zzudc.MW_Economic_Cell_EU,
                zzudc.MW_Primary_Packaging,
                zzudc.MW_Product_Form,
                zzudc.MW_Secondary_Packaging,
                zzpack.Pack_Family,
                zzpack.Pack_Sub_Family,
                zzraw.Physical_Condition,
                zzraw.Raw_Family,
                zzraw.Raw_Group,
                zzraw.Raws_Sub_Family,
                zzglobal.ID_Brand_Flag,
                zzglobal.ID_Business_Segment,
                zzglobal.ID_Market_Segment,
                zzglobal.ID_Product_Category,
                zzglobal.ID_Product_Type,
                tax.IT_CoPack_Group_ID,
                tax.Copack_Group,
                tax.IT_Pack_Size_ID,
                tax.Pack_Size,
                tax.IT_Tech_ID,
                tax.Tech,
                tax.IT_IntExt_Mfg_Group_ID,
                tax.IntExt_Mfg_Group,
                tax.IT_EC_Group_ID,
                tax.EC_Group,
                tax.IT_Financial_Product_Segment_ID,
                tax.Financial_Product_Segment,
                CASE WHEN LOWER(TRIM(sc.SievoCategoryL1)) IN ('', 'null', 'na', 'n/a', 'blank/s' ) OR sc.SievoCategoryL1 IS NULL THEN 'Missing' ELSE sc.SievoCategoryL1 END as SievoCategoryL1,
                CASE WHEN LOWER(TRIM(sc.SievoCategoryL2)) IN ('', 'null', 'na', 'n/a', 'blank/s' ) OR sc.SievoCategoryL2 IS NULL THEN 'Missing' ELSE sc.SievoCategoryL2 END as SievoCategoryL2,
                CASE WHEN LOWER(TRIM(sc.SievoCategoryL3)) IN ('', 'null', 'na', 'n/a', 'blank/s' ) OR sc.SievoCategoryL3 IS NULL THEN 'Missing' ELSE sc.SievoCategoryL3 END as SievoCategoryL3,
                CASE WHEN LOWER(TRIM(sc.SievoCategoryL4)) IN ('', 'null', 'na', 'n/a', 'blank/s' ) OR sc.SievoCategoryL4 IS NULL THEN 'Missing' ELSE sc.SievoCategoryL4 END as SievoCategoryL4,
                CASE WHEN LOWER(TRIM(sc.SievoCategoryL5)) IN ('', 'null', 'na', 'n/a', 'blank/s' ) OR sc.SievoCategoryL5 IS NULL THEN 'Missing' ELSE sc.SievoCategoryL5 END as SievoCategoryL5,
                CASE WHEN LOWER(TRIM(sc.SievoCategoryL6)) IN ('', 'null', 'na', 'n/a', 'blank/s' ) OR sc.SievoCategoryL6 IS NULL THEN 'Missing' ELSE sc.SievoCategoryL6 END as SievoCategoryL6,
                CAST(NULL as STRING) as Box_Qty
    FROM        df_material m
    LEFT JOIN   mdg_classification_zzglobal01_decoded zzglobal
        ON      m.Material = zzglobal.matnr_zzglobal
    LEFT JOIN   mdg_classification_zzpack01_decoded zzpack
        ON      m.Material = zzpack.matnr_zzpack
    LEFT JOIN   mdg_classification_zzraw01_decoded zzraw
        ON      m.Material = zzraw.matnr_zzraw
    LEFT JOIN   mdg_classification_zzudc_decoded zzudc
        ON      m.Material = zzudc.matnr_zzudc
    LEFT JOIN   mdg_regional_codes regio_codes
        ON      m.Material = regio_codes.Material
    LEFT JOIN   df_helios_material_taxonomy tax
        ON      m.Material = tax.Material
    LEFT JOIN   mdg_material_group mg
        ON      m.Material_Group = mg.Material_Group
    LEFT JOIN   df_manufacturing_type_per_material mft
        ON      mft.Material_Number = m.Material
    LEFT JOIN   df_sourcing_plant_per_material sp 
        ON      m.Material = sp.Material_Number
    LEFT JOIN   atlas_sievo_categories sc 
        ON      m.Material = sc.MaterialNumber
""").createOrReplaceTempView("int_material")

df=spark.sql("""
    WITH MARS_SEGMENT_DIVISION AS (
        SELECT  VALUE1 as Business_Segment,
                VALUE2 as Brand_Flag,
                RESULT1 as Mars_Segment,
                RESULT2 as Mars_Division
        FROM    v_odsrules
        WHERE   Function = "MDG_MARS_SEGMENT_AND_DIVISION"
            AND Criteria = "MARS_SEGMENT_DIVISION"
            AND COALESCE(VALUE2,"[Any/Other]") <> "[Any/Other]"
    ),
    MARS_SEGMENT_DIVISION_NO_BRAND AS (
        SELECT  VALUE1 as Business_Segment,
                RESULT1 as Mars_Segment,
                RESULT2 as Mars_Division
        FROM    v_odsrules
        WHERE   Function = "MDG_MARS_SEGMENT_AND_DIVISION"
            AND Criteria = "MARS_SEGMENT_DIVISION"
            AND COALESCE(VALUE2,"[Any/Other]") = "[Any/Other]"
    ),
    MARS_NET_WEIGHT_GRAMS AS (
        SELECT  VALUE1 as Material,
                RESULT1 as Net_Weight_override,
                CAST(RESULT2 AS STRING) as RuleM1,
                CAST(RESULT3 AS STRING) as override_flag
        FROM    v_odsrules
        WHERE   Function = "MDG_MARS_NET_WEIGHT_GRAMS"
            AND Criteria = "MARS_NET_WEIGHT_GRAMS"
    ),
    MDG_MARKET_SEGMENT AS (
        SELECT  DISTINCT
                VALUE1 as Material,
                RESULT1 as Market_Segment,
                CAST(RESULT2 AS STRING) as RuleM2,
                CAST(RESULT3 AS STRING) as override_flag
        FROM    v_odsrules
        WHERE   Function = "MDG_MARKET_SEGMENT"
            AND Criteria = "BY_MATERIAL"
    )

    SELECT      m.*,
                IF(w.RuleM1 IS NULL, m.Net_Weight_Grams, w.Net_Weight_override)       AS Net_Weight_Grams_Final,
                IF(w.RuleM1 IS NULL, m.Net_Weight, w.Net_Weight_override)             AS Net_Weight_Final,
                COALESCE(ms.Market_Segment, m.Market_Segment) AS Market_Segment_Final,
                COALESCE(sd.Mars_Segment,sdb.Mars_Segment) as Mars_Segment,
                COALESCE(sd.Mars_Division,sdb.Mars_Division) as Mars_Division,
                COALESCE(ms.override_flag,w.override_flag) as override_flag,
                CONCAT_WS(',',    
                    IF(w.RuleM1 IS NULL, NULL,
                        CONCAT(w.RuleM1, ':(Net_Weight = ', COALESCE(CAST( ROUND(m.Net_Weight, 2) AS STRING),'') ,'->', COALESCE(CAST( ROUND(w.Net_Weight_override, 2) AS STRING),''), ')')),
                    IF(ms.RuleM2 IS NULL, NULL,
                        CONCAT(ms.RuleM2, ':(Market_Segment = ', COALESCE(CAST(m.Market_Segment AS STRING),'') ,'->', COALESCE(CAST(ms.Market_Segment AS STRING),''), ')'))
                ) AS override_rule
    FROM        int_material m
    LEFT JOIN   MDG_MARKET_SEGMENT ms 
        ON      ms.Material = m.Material
    LEFT JOIN   MARS_SEGMENT_DIVISION sd 
        ON      sd.Business_Segment = m.Business_Segment
        AND     sd.Brand_Flag = m.Brand_Flag
    LEFT JOIN   MARS_SEGMENT_DIVISION_NO_BRAND sdb
        ON      sdb.Business_Segment = m.Business_Segment
    LEFT JOIN   MARS_NET_WEIGHT_GRAMS w 
        ON      w.Material = m.Material
""").withColumns({
        "Net_Weight_Grams" :            F.col("Net_Weight_Grams_Final"),
        "Net_Weight" :                  F.col("Net_Weight_Final"),
        "Market_Segment" :              F.col("Market_Segment_Final"),
        "SourceSystemUnit" :            F.lit(core_source_system_id).cast("string"),
        "src_agnostic_unique_ID" :      F.col("Material").cast("string"),
        "_run_id":                      F.lit(job_id),
        "_run_timestamp" :              F.current_timestamp()
    }) \
    .drop("Net_Weight_Grams_Final", "Net_Weight_Final", "Market_Segment_Final")


# Reorder columns
priority_cols = ['SourceSystemUnit', 'src_agnostic_unique_ID']
other_cols = [col for col in df.columns if col not in priority_cols]
df = df.select(priority_cols + other_cols)

## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

### dim_material <u>Source Agnostic</u> - UK Kind (Source System/Unit : 2)

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema":"output", "target_table":"dim_material", "format":"delta","opertation_type":"Managed Table Type - Append"},
    "source_tables": [
        {"schema":"normalized",     "view":"ukkind_item",           "table":"ukkind_item"},
        {"schema":"raw",            "view":"ukkind_salesdetail",    "table":"ukkind_salesdetail"},
        {"schema":"output",         "view":"dim_material",          "table":"dim_material"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

print(f"uk_kind_source_system_id: {uk_kind_source_system_id}")

uk_kind_dim_material_df = spark.sql("""
    SELECT DISTINCT
        a.ItemNumber                                                AS Material,
        CONCAT(a.FlavorDescription, ' - ', a.ItemDescription2)      AS Description,
        'FERT'                                                      AS Material_Type,
        NULL                                                        AS IsComan,
        NULL                                                        AS IsCopack,
        False                                                       AS IsInhouse,
        CAST(0 AS Double)                                           AS Gross_Weight,
        '0'                                                         AS Net_Weight,
        'G'                                                         AS Weight_Unit,
        '1'                                                         AS Quantity_of_1,
        CAST(a.ItemUPC AS LONG)                                     AS EAN,
        'EA'                                                        AS Base_Unit_of_Measure,
        'X'                                                         AS Traded_Unit,
        CONCAT('KIND', ' - ', a.Brand)                              AS Brand_Essence,
        a.ItemConfiguration                                         AS Traded_Unit_Configuration,
        'Chocolate'                                                 AS Business_Segment,                      
        CONCAT('KIND', ' - ', a.Brand)                              AS Brand_Flag,
        'Snacking'                                                  AS Mars_Segment,
        'Health & Wellness'                                         AS Mars_Division,
        '2'                                                         AS SourceSystemUnit,
        CONCAT('2', ':', a.ItemNumber)                              AS src_agnostic_unique_ID
    FROM        ukkind_item a
    INNER JOIN  ukkind_salesdetail b                                                                    -- filter the data only to materials that appear in sales data
        ON      a.ItemNumber                                        =  b.ItemNumber
    WHERE       a.ProductType                                       IN ('Assembly', 'Product')          -- only take finished goods
""").withColumns({
        "_run_id": F.lit(job_id),
        "_run_timestamp" :  F.current_timestamp(),
        "SourceSystemUnit": F.lit(uk_kind_source_system_id).cast("string"),
        "src_agnostic_unique_ID": F.concat(F.lit(uk_kind_source_system_id), F.lit(":"), F.col("Material")).cast("string")
    })



# Cast all columns from uk_kind_dim_material_df to match the exact data types in dim_material
# This ensures schema compatibility and prevents type mismatch errors during the append operation
dim_material_df = spark.table("dim_material")
df = cast_dataframe_to_schema(uk_kind_dim_material_df, dim_material_df)


## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

### dim_material <u>Source Agnostic</u> - Fusion (Source System/Unit : 3)

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema":"output", "target_table":"dim_material", "format":"delta","opertation_type":"Managed Table Type - Append"},
    "source_tables": [
        {"schema":"raw",            "view":"fusion_dim_material",       "table":"fusion_dim_material"},
        {"schema":"output",         "view":"dim_material",              "table":"dim_material"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

print(f"fusion_source_system_id: {fusion_source_system_id}")


fusion_dim_material_df = spark.sql("""
    SELECT      material_number as Material ,
                material_description as Description ,
                material_internal_code as Representative_Item ,

                -- as Veritas_Spec_Number ,

                material_type as Material_Type ,
                CAST(NULL as BOOLEAN) as IsComan ,
                CAST(NULL as BOOLEAN) as IsCopack ,
                False as IsInhouse ,
                cross_plant as Sourcing_Plant ,
                external_material_group as Material_Group_Long_Description ,

                --as Created_By,
                 
                source_record_modify_date as Created_On,
                gross_weight as Gross_Weight ,
                CASE    WHEN weight_unit='KG' THEN net_weight * 1000.00
                        WHEN weight_unit='G'  THEN net_weight
                        WHEN weight_unit='LB' THEN net_weight * 453.59237
                        WHEN weight_unit='OZ' THEN net_weight * 28.3495231
                        WHEN weight_unit='MG' THEN net_weight / 1000.00
                        WHEN weight_unit='TO' THEN net_weight * 1000000.00
                        ELSE weight_unit END                                          AS Net_Weight_Grams,
                net_weight as Net_Weight ,
                weight_unit as Weight_Unit ,
                plant_bulk_status_code as X_Plant_Status ,

                -- as X_Plant_Status_Desc ,

                1 as Quantity_of_1,
                length as Length ,
                width as Width ,
                height as Height ,
                ean as EAN ,
                ean_category as EAN_Category,
                unit_of_dimension as Unit_of_Dimension,
                package_material_group as Material_Group,
                unit_of_measure as Base_Unit_of_Measure,

                -- as Document ,

                total_shelf_life as Total_shelf_life ,
                min_remng_shelf_life as Min_Rem_Shelf_Life ,
                inner_pack_quantity as Merchandising_Unit ,

                --as Retail_Sales_Unit ,

                po_unit_of_measure as Traded_Unit ,

                -- as Animal_Life_Stage ,
                -- as Animal_Breed ,
                -- as Animal_Parts ,
                -- as Animal_Size ,
                -- as Animal_Species ,

                story as Brand_Essence ,

                -- as CoManufact_Packing_Activity ,
                -- as CoPacking_Packing_Activity ,
                -- as CoPacking_Product_Format ,
                -- as CoPacking_Technology ,
                -- as Compliant_for_Countries ,
                -- as Cuisine ,
                -- as Diet_Claims ,
                -- as Display_Storage_Condition ,
                -- as Lifestyle ,
                -- as On_pack_Consumer_Offer ,
                -- as On_pack_Consumer_Value ,
                -- as On_pack_Trade_Offer ,
                -- as Product_Pack_Size_Group ,
                -- as Sustainability ,
                casepack_quantity as Traded_Unit_Configuration ,
                -- as Traded_Unit_Format ,
                division  as Business_Segment ,
                -- as Market_Segment ,
                -- as Brand_Flag ,
                -- as Brand_Sub_Flag ,
                -- as Supply_Segment ,
                -- as Ingredient_Variety ,
                -- as Functional_Variety ,
                indust_sectr as Trade_Sector ,
                -- as Marketing_Concept ,
                -- as Multipack_Quantity ,
                -- as Occasion ,
                -- as Product_Category ,
                -- as Product_Type ,
                size as Product_Pack_Size ,
                -- as Consumer_Pack_Type ,
                -- as Consumer_Pack_Format ,
                -- as FPPS_Minor_Pack ,
                -- as US_Value_Chain_Segment ,
                -- as MW_Economic_Cell_EU ,
                -- as MW_Primary_Packaging ,
                -- as MW_Product_Form ,
                -- as MW_Secondary_Packaging ,
                -- as Pack_Family ,
                -- as Pack_Sub_Family ,
                -- as Physical_Condition ,
                -- as Raw_Family ,
                basic_material as Raw_Group ,
                -- as Raws_Sub_Family ,
                -- as ID_Brand_Flag ,
                -- as ID_Business_Segment ,
                -- as ID_Market_Segment ,
                -- as ID_Product_Category ,
                -- as ID_Product_Type ,
                -- as IT_CoPack_Group_ID ,
                -- as Copack_Group ,
                -- as IT_Pack_Size_ID ,
                -- as Pack_Size ,
                -- as IT_Tech_ID ,
                -- as Tech ,
                -- as IT_IntExt_Mfg_Group_ID ,
                -- as IntExt_Mfg_Group ,
                -- as IT_EC_Group_ID ,
                -- as EC_Group ,
                -- as IT_Financial_Product_Segment_ID ,
                -- as Financial_Product_Segment ,

                'N/A' as SievoCategoryL1 ,
                'N/A' as SievoCategoryL2 ,

                division_name as SievoCategoryL3 ,
                department_name as SievoCategoryL4 ,
                class_name as SievoCategoryL5 ,
                sub_class_name as SievoCategoryL6 


                -- as Box_Qty ,
                -- as Mars_Segment ,
                -- as Mars_Division ,
                -- as override_flag ,
                -- as override_rule ,
                -- as SourceSystemUnit ,
                -- as src_agnostic_unique_ID
    FROM        fusion_dim_material
""").withColumns({
        "_run_id": F.lit(job_id),
        "_run_timestamp" :  F.current_timestamp(),
        "SourceSystemUnit": F.lit(fusion_source_system_id).cast("string"),
        "src_agnostic_unique_ID": F.concat(F.lit(fusion_source_system_id), F.lit(":"), F.col("Material")).cast("string")
    })



# Cast all columns from fusion_dim_material to match the exact data types in dim_material
# This ensures schema compatibility and prevents type mismatch errors during the append operation
dim_material_df = spark.table("dim_material")
df = cast_dataframe_to_schema(fusion_dim_material_df, dim_material_df)

## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

## dim_vendor

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "dim_vendor", "format":"delta","opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
        {"schema":"normalized",         "table":"vendor_text",                  "view":"vendor_text",                   "project_name":"atlas"},
        {"schema":"normalized",         "table":"vendor_attributes",            "view":"vendor_attributes",             "project_name":"atlas"},
        
        {"schema":"raw",                "view":"fusion_dim_vendor",     "table":"fusion_dim_vendor"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_schema_name": "'''+str(admin_schema_name)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''",
    "process_date": "'''+str(process_date)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

#   CORE
core_vendor_df = spark.sql("""
    SELECT      a.Account_Number                                AS supplier_number,
                a.Account_Name                                  AS supplier_name,
                b.Country_Key                                   AS supplier_country,
                b.Supplier_Account_Group                        AS supplier_account_group,
                b.City                                          AS city,
                b.VAT_Registration_Number                       AS vat_registration_number,
                b.Central_Deletion_Flag_for_Master_Record       AS deletion_flag
    FROM        vendor_text a
    LEFT JOIN   vendor_attributes b
        ON      a.Account_Number = b.Account_Number
""").withColumns({
        "SourceSystemUnit": F.lit(core_source_system_id).cast("string"),
        "src_agnostic_unique_ID": F.col("supplier_number").cast("string")
    })
 

#   FUSION
fusion_vendor_df = spark.sql("""
    SELECT      vendor_code                         AS supplier_number,
                vendor_name                         AS supplier_name,
                country_key                         AS supplier_country,
                city                                AS city,
                tax_number                          AS vat_registration_number
    FROM        fusion_dim_vendor a

""").withColumns({
        "SourceSystemUnit": F.lit(fusion_source_system_id).cast("string"),
        "src_agnostic_unique_ID": F.concat(F.lit(fusion_source_system_id), F.lit(":"), F.col("supplier_number")).cast("string")
    })
 

df = core_vendor_df.unionByName(fusion_vendor_df, allowMissingColumns=True).withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

# Reorder columns
priority_cols = ['SourceSystemUnit', 'src_agnostic_unique_ID']
other_cols = [col for col in df.columns if col not in priority_cols]
df = df.select(priority_cols + other_cols)

## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

## dim_customer

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output",     "target_table": "dim_customer", "format":"delta","opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
        {"schema":"normalized",     "table":"customer_attributes",          "view":"customer_attributes",       "project_name":"atlas"},
        {"schema":"raw",            "table":"ukkind_customer",              "view":"ukkind_customer"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_schema_name": "'''+str(admin_schema_name)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''",
    "process_date": "'''+str(process_date)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

#   CORE
df_cust_atlas = spark.sql("""
    SELECT  a.Customer_Number,
            a.Name_1,
            a.Address,
            a.Central_Deletion_Flag_for_Master_Record,
            a.City as City,
            a.City_Code,
            a.Country_Key,
            a.Customer_Account_Group,
            a.Date_on_which_the_Record_Was_Created,
            a.District,
            a.Name_2,
            a.Postal_Code,
            a.Region
    FROM    customer_attributes a
""").withColumns({
        "SourceSystemUnit": F.lit(core_source_system_id).cast("string"),
        "src_agnostic_unique_ID": F.col("Customer_Number").cast("string")
    })

#   UK KIND
df_cust_ukkind = spark.sql("""
    SELECT  a.TempCustomerId                                AS Customer_Number,
            a.Name                                          AS Name_1,
            a.CustomerType                                  AS Customer_Account_Group
    FROM    ukkind_customer a
""").withColumns({
        "SourceSystemUnit": F.lit(uk_kind_source_system_id).cast("string"),
        "src_agnostic_unique_ID": F.concat(F.lit(uk_kind_source_system_id), F.lit(":"), F.col("Customer_Number")).cast("string")
    })


# Union data
df = df_cust_atlas.unionByName(df_cust_ukkind, allowMissingColumns=True) \
                  .withColumns({
                          "_run_id": lit(job_id),
                          "_run_timestamp" :  F.current_timestamp()
                      })

# Reorder columns
priority_cols = ['SourceSystemUnit', 'src_agnostic_unique_ID']
other_cols = [col for col in df.columns if col not in priority_cols]
df = df.select(priority_cols + other_cols)

## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

## dim_company_code

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "dim_company_code", "format":"delta","opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
        {"schema":"normalized",         "table":"company_code_text",            "view":"company_code_text",                     "project_name":"atlas"},
        {"table":"dbo.v_odsrules",      "view":"v_odsrules",                    "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"},
        {"schema":"raw",                "table":"fusion_fact_purchase_order_header",            "view":"fusion_fact_purchase_order_header"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_schema_name": "'''+str(admin_schema_name)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''",
    "process_date": "'''+str(process_date)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

#   CORE
df_company_atlas = spark.sql("""
    SELECT          a.*
    FROM            company_code_text a
""").withColumns({
        "SourceSystemUnit": F.lit(core_source_system_id).cast("string"),
        "src_agnostic_unique_ID": F.col("Company_Code").cast("string")
    })

#   UK KIND
df_company_ukkind = spark.sql("""
    SELECT  value2                                      AS Company_Code,
            value3                                      AS Company_Name,
            value1                                      AS SourceSystemUnit,
            CONCAT(value1, ':', value2)                 AS src_agnostic_unique_ID
    FROM    v_odsrules
    WHERE   function = 'SYSTEM_AGNOSTIC_SETUP'
        AND criteria = 'COMPANY_CODE'
""")

#   FUSION
df_company_fusion = spark.sql("""
    SELECT  DISTINCT
            company_code                                AS Company_Code,
            company_code                                AS Company_Name
    FROM    fusion_fact_purchase_order_header

""").withColumns({
        "SourceSystemUnit": F.lit(fusion_source_system_id).cast("string"),
        "src_agnostic_unique_ID": F.concat(F.lit(fusion_source_system_id), F.lit(":"), F.col("Company_Code")).cast("string")
    })


# Union data
df = df_company_atlas.unionByName(df_company_ukkind, allowMissingColumns=True) \
                     .unionByName(df_company_fusion, allowMissingColumns=True) \
                     .withColumns({
                             "_run_id": lit(job_id),
                             "_run_timestamp" :  F.current_timestamp()
                         })

# Reorder columns
priority_cols = ['SourceSystemUnit', 'src_agnostic_unique_ID']
other_cols = [col for col in df.columns if col not in priority_cols]
df = df.select(priority_cols + other_cols)


## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

## dim_sales_organization

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "dim_sales_organization", "format":"delta","opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
        {"schema":"normalized",    "table":"sales_organization",        "view":"sales_organization",                    "project_name":"atlas"},
        {"schema":"normalized",    "table":"sales_organization_dict",   "view":"sales_organization_dict",               "project_name":"atlas"},
        {"table":"dbo.v_odsrules", "view":"v_odsrules",                 "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_schema_name": "'''+str(admin_schema_name)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''",
    "process_date": "'''+str(process_date)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

#   CORE
df_atlas = spark.sql("""

    SELECT      s.sales_organization,
                d.description,
                s.company_code,
                s.country_code,
                s.currency,
                s.customer_number                             
    FROM        sales_organization s
    LEFT JOIN   sales_organization_dict d 
        ON      s.sales_organization = d.sales_organization
""").withColumns({
        "SourceSystemUnit": F.lit(core_source_system_id).cast("string"),
        "src_agnostic_unique_ID": F.col("sales_organization").cast("string")
    })


#   UK KIND
df_system_agnostic = spark.sql("""
    SELECT  value2                                      AS sales_organization,
            value3                                      AS description,
            value4                                      AS country_code,
            value5                                      AS company_code,
            value1                                      AS SourceSystemUnit,
            CONCAT(value1, ':', value2)                 AS src_agnostic_unique_ID
    FROM    v_odsrules
    WHERE   function = 'SYSTEM_AGNOSTIC_SETUP'
        AND criteria = 'SALES_ORGANIZATION'
""")

df_joined = df_atlas.unionByName(df_system_agnostic, allowMissingColumns=True).withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df_joined)

## dim_calendar_day

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "dim_calendar_day", "format":"delta","opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
            {"schema":"raw", "table": "mars_calendar", "view": "mars_calendar"}
        ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

df = spark.sql("""
    SELECT  DISTINCT
            md5(GCALDAY)                                                AS id_calendar,
            DATE_FORMAT(TO_DATE(GCALDAY, 'yyyyMMdd'), 'yyyy-MM-dd')     AS day,
            CAST(GCALYEAR AS INT)                                       AS year,
            GMARSPINY___T                                               AS period,
            GMARSQINY___T                                               AS quarter,
            GMARSWINP___T                                               AS week,
            GMARSYP                                                     AS year_period,
            GMARSYPW                                                    AS year_period_week
    FROM    mars_calendar
    WHERE   DATE_FORMAT(TO_DATE(GCALDAY, 'yyyyMMdd'), 'yyyy-MM-dd')     IS NOT NULL

""").withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

## dim_calendar_period

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "dim_calendar_period", "format":"delta","opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
            {"schema":"raw", "table": "mars_calendar", "view": "mars_calendar"}
        ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

df = spark.sql("""
    SELECT 
                md5(CONCAT_WS("",GCALYEAR,GMARSPINY___T)) as id_calendar,
                cast(GCALYEAR as int) as year,
                GMARSPINY___T as period,
                GMARSQINY___T as quarter,
                MIN(TO_DATE(GCALDAY, 'yyyyMMdd')) as period_first_day,
                MAX(TO_DATE(GCALDAY, 'yyyyMMdd')) as period_last_day,
                CONCAT_WS("",GCALYEAR,GMARSPINY___T) as year_period
    FROM        mars_calendar
    GROUP BY    GCALYEAR,
                GMARSPINY___T,
                GMARSQINY___T,
                GCALYEAR
""").withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

## dim_calendar_quarter

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "dim_calendar_quarter", "format":"delta","opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
            {"schema":"raw", "table": "mars_calendar", "view": "mars_calendar"}
        ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

df = spark.sql("""
    SELECT 
                md5(CONCAT_WS("",GCALYEAR,GMARSQINY___T)) as id_calendar,
                cast(GCALYEAR as int) as year,
                GMARSQINY___T as quarter,
                MIN(TO_DATE(GCALDAY, 'yyyyMMdd')) as quarter_first_day,
                MAX(TO_DATE(GCALDAY, 'yyyyMMdd')) as quarter_last_day,
                CONCAT_WS("",GCALYEAR,GMARSQINY___T) as year_quarter
    FROM        mars_calendar
    GROUP BY    GCALYEAR,
                GMARSQINY___T,
                GCALYEAR
""").withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

## dim_component

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema":"output",  "target_table":"dim_component",     "format":"delta",   "opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
        {"schema":"output",        "table":"dim_material",     "view":"dim_material"},
        {"table":"dbo.cr579_loraxpackfamilytopackgroup",        "view":"cr579_loraxpackfamilytopackgroup",  "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"},
        {"table":"dbo.GlobalOptionsetMetadata",                 "view":"GlobalOptionsetMetadata",           "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"}        
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

component_df = spark.sql(f"""
    SELECT      Material,
                Material_Type,
                Base_Unit_of_Measure,
                Brand_Flag,
                Brand_Sub_Flag,
                Created_On,
                Description,
                Net_Weight,
                Net_Weight_Grams,
                Pack_Family,
                Pack_Sub_Family,
                Quantity_of_1,
                Weight_Unit,
                X_Plant_Status,
                X_Plant_Status_Desc,
                Created_By,
                Document,
                CAST(EAN AS string) as EAN,
                EAN_Category,
                Material_Group,
                Unit_of_Dimension,
                Length,
                Width,
                Height,
                Veritas_Spec_Number as Regional_Code_Number,
                Raw_Family,
                Raw_Group,
                Raws_Sub_Family,
                Length * Width as Area,
                Length * Width * Height as Volume,
                g_usage.LocalizedLabel as Usage,
                g_packgroup.LocalizedLabel as Pack_Group,
                g_main.LocalizedLabel as Main,
                g_flexrigid.LocalizedLabel as Flex_Rigid,
                g_disposalcodegeneric.LocalizedLabel as Disposal_Code_Generic,
                '1'                                         AS SourceSystemUnit,
                m.Material                                  AS src_agnostic_unique_ID
    FROM        dim_material m
    LEFT JOIN   cr579_loraxpackfamilytopackgroup l
        ON      UPPER(m.Pack_Family)                        = UPPER(l.cr579_packfamilyname)
        AND     UPPER(m.Pack_Sub_Family)                    = UPPER(l.cr579_packsubfamilyname)
    LEFT JOIN   GlobalOptionsetMetadata g_flexrigid 
        ON      UPPER(l.cr579_flexrigid)                    = UPPER(g_flexrigid.Option) 
        AND     UPPER(g_flexrigid.OptionSetName)            = UPPER('cr579_flexrigid')
    LEFT JOIN   GlobalOptionsetMetadata g_main 
        ON      UPPER(l.cr579_main)                         = UPPER(g_main.Option)
        AND     UPPER(g_main.OptionSetName)                 = UPPER('cr579_main')
    LEFT JOIN   GlobalOptionsetMetadata g_packgroup
        ON      UPPER(l.cr579_packgroup)                    = UPPER(g_packgroup.Option) 
        AND     UPPER(g_packgroup.OptionSetName)            = UPPER('cr579_packgroup')
    LEFT JOIN   GlobalOptionsetMetadata g_usage
        ON      UPPER(l.cr579_usage)                        = UPPER(g_usage.Option)
        AND     UPPER(g_usage.OptionSetName)                = UPPER('cr579_usage')
    LEFT JOIN   GlobalOptionsetMetadata g_disposalcodegeneric
        ON      UPPER(l.lk_disposalcodegeneric)             = UPPER(g_disposalcodegeneric.Option)
        AND     UPPER(g_disposalcodegeneric.OptionSetName)  = UPPER('lk_disposalcodegeneric')
    WHERE       Material_Type IN ('VERP', 'UNBW', 'ZQFC')
""").withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, component_df)

## dim_component_hybrid

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema":"output", "target_table":"dim_component_hybrid", "format":"delta","opertation_type":"Managed Table Type", "params":{"drop_table":"True"}},
    "source_tables": [
        {"schema":"output",    "table":"dim_component",     "view":"dim_component"},
        {"view_name":"material_spec_finder",        "path":"environments/veripack/prod/vrp_process/material_spec_finder",    "container":"synapse" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service"},
        {"view_name":"shared_dim_pms_veripack",     "path":"environments/veripack/prod/vrp_process/shared_dim_pms_veripack", "container":"synapse" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service"},
        {"view_name":"packaging_layers",            "path":"environments/veripack/prod/vrp_output/packaging_layers",         "container":"synapse" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service"},

        {"table":"dbo.lk_componentoverride",            "view":"lk_componentoverride",              "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"},
        {"table":"dbo.GlobalOptionsetMetadata",         "view":"GlobalOptionsetMetadata",           "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"},
        {"table":"dbo.cr579_loraxpackfamilytopackgroup","view":"cr579_loraxpackfamilytopackgroup",  "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"},
        {"table":"dbo.cr579_pacinnermaterial",          "view":"cr579_pacinnermaterial",            "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

# ======================================================================================================================================================
# =====================                                     1. Core ODS process                                                    =====================
# ======================================================================================================================================================

from pyspark.sql import functions as F

spark.sql("""
    SELECT      DISTINCT
                p.Category,
                p.SubCategory,
                CASE WHEN p.TARE_WEIGHT_GRAMS = 0.0 THEN NULL ELSE p.TARE_WEIGHT_GRAMS END as TARE_WEIGHT_GRAMS,
                p.Usage_final,
                p.Packaging_Group,
                p.Disposal_Code_Final,
                p.Main_Final,
                p.Height_value,
                p.Width_value,
                p.Depth_value,
                p.BASISWEIGHTTOTAL                  AS basisweighttotal,
                p.Bio_Based                         AS bio_based,
                p.Branded                           AS branded,
                p.BUSINESSUNITS                     AS businessunits,
                p.Compostability_Certificate        AS compostability_certificate,
                p.Consumer_ReUse                    AS consumer_reuse,
                p.D4C_Category                      AS d4c_category,
                p.D4C_Design                        AS d4c_design,
                p.DIAMETER                          AS diameter,
                p.DiameterUOM                       AS diameteruom,
                p.DIRECTFOODCONTACT                 AS directfoodcontact,
                p.ENVIRONMENTALPACKAGINGTYPE        AS environmentalpackagingtype,
                p.EXTERNALVOLUMEMETRICWIDTHVALUE    AS external_volume,
                p.Fillers_Increase_Density          AS fillers_increase_density,
                p.GROUP                             AS group,
                p.Is_Paper_Cert_Ver_Recy            AS is_paper_cert_ver_recy,
                p.LabelWaterSol                     AS labelwatersol,
                p.LACQUERVARNISH                    AS lacquervarnish,
                p.LAMINATE                          AS laminate,
                p.LAMINATIONSYSTEM                  AS laminationsystem,
                p.MANUFACTURINGPROCESS              AS manufacturingprocess,
                p.Material_Color                    AS material_color,
                p.Material_Thickness                AS thickness,
                p.MATERIALTHICKNESSAVERAGEMETRIC    AS thickness_avg,
                p.Metalized                         AS metalized,
                p.Mono_Multi                        AS mono_multi,
                p.PACKAGINGITEMTYPE                 AS packagingitemtype,
                p.PackReason_Change                 AS packreason_change,
                p.PackSpec_Current_Owner            AS packspec_current_owner,
                p.PackSpec_Originator               AS packspec_originator,
                p.PLASTICMASTERBATCHCOLOURREF       AS plasticmasterbatchcolourref,
                p.Problematic_Materials             AS problematic_materials,
                p.Recycled_P                        AS percentrecyclable,
                p.Reinforcement                     AS reinforcement,
                p.SEALTYPE                          AS sealtype,
                p.Sleeve_Design                     AS sleeve_design,
                p.SPECNAME                          AS specname,
                p.SPECNUMBER                        AS specnumber,
                p.SPECSTATUS                        AS specstatus,
                p.SURFACEAREAUNITMETRIC             AS surfaceareaunitmetric,
                p.SurfaceUOM                        AS surfaceuom,
                p.Sustainable_Source_Certificate    AS sustainable_source_certificate,
                p.ThicknessAvgUOM                   AS thickness_avg_uom,
                p.ThicknessUOM                      AS thickness_uom,
                p.TOTALSHELFLIFE                    AS totalshelflife,
                p.Type_of_Recycled_Content          AS type_of_recycled_content,
                p.VolumeUOM                         AS external_volume_uom,
                p.Water_Based_Adhesive              AS water_based_adhesive,
                p.DimensionUOM,
                p.Flex_Rigid,
                p.Packaging_Group,
                pl.Layer_Category_1                 AS layer_category_1, 
                pl.Layer_Category_2                 AS layer_category_2, 
                pl.Layer_Category_3                 AS layer_category_3, 
                pl.Layer_Category_4                 AS layer_category_4, 
                pl.Layer_Category_5                 AS layer_category_5, 
                pl.Layer_Category_6                 AS layer_category_6, 
                pl.Layer_Category_7                 AS layer_category_7, 
                pl.Layer_Category_8                 AS layer_category_8, 
                pl.Layer_Category_9                 AS layer_category_9, 
                pl.`FLEXIBLECOMP1%`                 AS percentflexiblecomp1, 
                pl.`FLEXIBLECOMP2%`                 AS percentflexiblecomp2, 
                pl.`FLEXIBLECOMP3%`                 AS percentflexiblecomp3, 
                pl.`FLEXIBLECOMP4%`                 AS percentflexiblecomp4, 
                pl.`FLEXIBLECOMP5%`                 AS percentflexiblecomp5, 
                pl.`FLEXIBLECOMP6%`                 AS percentflexiblecomp6, 
                pl.`FLEXIBLECOMP7%`                 AS percentflexiblecomp7, 
                pl.`FLEXIBLECOMP8%`                 AS percentflexiblecomp8, 
                pl.`FLEXIBLECOMP9%`                 AS percentflexiblecomp9,
                pl.layer_indicator,
                p.INNERLINERMATERIAL                AS innerlinermaterial          
    FROM        shared_dim_pms_veripack p 
    LEFT JOIN   packaging_layers pl
        ON      p.SPECNUMBER = pl.SPECNUMBER 
        AND     pl.Total_Percentage_Weight_Tracking = 'Equals 100'
""").createOrReplaceTempView("shared_pms")

spark.sql("""
    SELECT      lk_componentname        AS material,
                lk_componentspecname    AS specnumber,
                lk_componentweightg     AS component_weight_g,
                lk_usagename            AS component_usage, 
                lk_categoryname         AS component_category, 
                lk_disposalcodename     AS component_disposalcode,
                lk_subcategoryname      AS component_subcategory,
                cr579_packgroupname     AS component_pack_group,
                ARRAY(
                    IF(lk_componentspecname  IS NOT NULL,'R1',NULL),
                    IF(lk_componentweightg   IS NOT NULL,'R2',NULL),
                    IF(lk_usagename          IS NOT NULL,'R3',NULL),
                    IF(lk_categoryname       IS NOT NULL,'R4',NULL),
                    IF(lk_disposalcodename   IS NOT NULL,'R5',NULL),
                    IF(lk_subcategoryname    IS NOT NULL,'R6',NULL),
                    IF(cr579_packgroupname   IS NOT NULL,'R7',NULL)
                ) AS rules_arr
    FROM        lk_componentoverride
    WHERE       statuscode = 1
""").createOrReplaceTempView("component_override")

spark.sql("""
    SELECT      c.Material                          AS material,
                c.Description                       AS component_description,
                c.Material_Type                     AS material_type,
                c.X_Plant_Status_Desc               AS material_status,
                c.EAN                               AS component_ean_upc_code,
                c.EAN_Category                      AS component_ean_category,
                COALESCE(p.Category, m.Category, c.Pack_Family)                                                         AS component_category,
                co.component_category                                                                                   AS component_category_override,
                COALESCE(P.SubCategory, m.SubCategory, c.Pack_Sub_Family)                                               AS component_subcategory,
                co.component_subcategory                                                                                AS component_subcategory_override,
                COALESCE(p.TARE_WEIGHT_GRAMS, c.Net_Weight_Grams)                                                       AS component_weight_g,
                co.component_weight_g                                                                                   AS component_weight_g_override,
                COALESCE(p.Height_value, m.Height_value, c.Height)                                                      AS component_height,
                COALESCE(p.Width_value, m.Width_value, c.Width)                                                         AS component_width,
                COALESCE(p.Depth_value, m.Depth_value, c.Length)                                                        AS component_depth, 
                COALESCE(p.DimensionUOM,c.Unit_of_Dimension)                                                            AS component_dimension_uom, 
                CASE WHEN p.Depth_value * p.Width_value IS NULL THEN c.Area ELSE p.Depth_value * p.Width_value END      AS component_area,
                CASE WHEN p.Depth_value * p.Width_value * p.Height_value IS NULL THEN c.Volume 
                    ELSE p.Depth_value * p.Width_value * p.Height_value END                                             AS component_volume,
                COALESCE(p.Flex_Rigid,c.Flex_Rigid)                                                                     AS component_flex_rigid, 
                COALESCE(p.Usage_final,m.Usage_final,c.Usage)                                                           AS component_usage,
                co.component_usage                                                                                      AS component_usage_override,
                COALESCE(p.Packaging_Group,m.Packaging_Group,c.Pack_Group)                                              AS component_pack_group,
                co.component_pack_group                                                                                 AS component_pack_group_override,
                COALESCE(p.Disposal_Code_Final,m.Disposal_Code_Final,c.Disposal_Code_Generic )                          AS component_disposalcode,
                co.component_disposalcode                                                                               AS component_disposalcode_override,
                COALESCE(p.Main_Final,m.Main_Final,c.Main)                                                              AS component_is_main,
                p.basisweighttotal,
                p.bio_based,
                p.branded,
                p.businessunits,
                p.compostability_certificate,
                p.consumer_reuse,
                p.d4c_category,
                p.d4c_design,
                p.diameter,
                p.diameteruom,
                p.directfoodcontact,
                p.environmentalpackagingtype,
                p.external_volume,
                p.fillers_increase_density,
                p.group,
                p.is_paper_cert_ver_recy,
                p.labelwatersol,
                p.lacquervarnish,
                p.laminate,
                p.laminationsystem,
                p.manufacturingprocess,
                p.material_color,
                p.thickness,
                p.thickness_avg,
                p.metalized,
                p.mono_multi,
                p.packagingitemtype,
                p.packreason_change,
                p.packspec_current_owner,
                p.packspec_originator,
                p.plasticmasterbatchcolourref,
                p.problematic_materials,
                p.percentrecyclable,
                p.reinforcement,
                p.sealtype,
                p.sleeve_design,
                p.specname,
                COALESCE(co.specnumber, p.SPECNUMBER) as specnumber,
                p.SPECNUMBER as Source_specnumber,
                p.specstatus,
                p.surfaceareaunitmetric,
                p.surfaceuom,
                p.sustainable_source_certificate,
                p.thickness_avg_uom,
                p.thickness_uom,
                p.totalshelflife,
                p.type_of_recycled_content,
                p.external_volume_uom,
                p.water_based_adhesive,
                p.layer_category_1, 
                p.layer_category_2, 
                p.layer_category_3, 
                p.layer_category_4, 
                p.layer_category_5, 
                p.layer_category_6, 
                p.layer_category_7, 
                p.layer_category_8, 
                p.layer_category_9, 
                p.percentflexiblecomp1, 
                p.percentflexiblecomp2, 
                p.percentflexiblecomp3, 
                p.percentflexiblecomp4, 
                p.percentflexiblecomp5, 
                p.percentflexiblecomp6, 
                p.percentflexiblecomp7, 
                p.percentflexiblecomp8, 
                p.percentflexiblecomp9,
                p.layer_indicator,
                m.More_Than_1_Spec                  AS more_than_1_spec,
                m.Matching_Method                   AS matching_method,
                p.innerlinermaterial,
                CASE WHEN co.material IS NOT NULL THEN 'X' ELSE CAST(NULL AS STRING) END AS override_flag,
                co.rules_arr
    FROM        dim_component c
    LEFT JOIN   component_override co 
        ON      co.material = c.material
    LEFT JOIN   material_spec_finder m 
        ON      c.Material = m.GRD_Code
    LEFT JOIN   shared_pms p
        ON      p.SPECNUMBER = COALESCE(co.specnumber,m.FINAL_SPECNUMBER,c.regional_code_number)  -- if override specnumber exists
""").createOrReplaceTempView("base")

df=spark.sql("""
    WITH base_ranked AS (
        SELECT
            base.*,
            ROW_NUMBER() OVER (
                PARTITION BY base.material 
                ORDER BY base.component_weight_g DESC, specnumber DESC
            ) AS rn,
            SUM(base.component_weight_g) OVER (PARTITION BY base.material) AS total_weight
        FROM base
    ), 
    component AS (
        SELECT  b.material,
                b.component_description,
                b.material_type,
                b.material_status,
                b.component_ean_upc_code,
                b.component_ean_category,
                COALESCE(b.component_category_override, b.component_category) AS component_category,
                COALESCE(b.component_subcategory_override, b.component_subcategory) AS component_subcategory,
                b.component_height,
                b.component_width,
                b.component_depth,
                b.component_dimension_uom,
                b.component_area,
                b.component_volume,
                b.component_flex_rigid,
                COALESCE(b.component_usage_override, b.component_usage) as component_usage,
                COALESCE(b.component_pack_group_override, b.component_pack_group) AS component_pack_group,
                COALESCE(b.component_disposalcode_override, b.component_disposalcode) AS component_disposalcode,
                b.component_is_main,
                b.basisweighttotal,
                b.bio_based,
                b.branded,
                b.businessunits,
                b.compostability_certificate,
                b.consumer_reuse,
                b.d4c_category,
                b.d4c_design,
                b.diameter,
                b.diameteruom,
                b.directfoodcontact,
                b.environmentalpackagingtype,
                b.external_volume,
                b.fillers_increase_density,
                b.group,
                b.is_paper_cert_ver_recy,
                b.labelwatersol,
                b.lacquervarnish,
                b.laminate,
                b.laminationsystem,
                b.manufacturingprocess,
                b.material_color,
                b.thickness,
                b.thickness_avg,
                b.metalized,
                b.mono_multi,
                b.packagingitemtype,
                b.packreason_change,
                b.packspec_current_owner,
                b.packspec_originator,
                b.plasticmasterbatchcolourref,
                b.problematic_materials,
                b.percentrecyclable,
                b.reinforcement,
                b.sealtype,
                b.sleeve_design,
                b.specname,
                b.specnumber,
                b.specstatus,
                b.surfaceareaunitmetric,
                b.surfaceuom,
                b.sustainable_source_certificate,
                b.thickness_avg_uom,
                b.thickness_uom,
                b.totalshelflife,
                b.type_of_recycled_content,
                b.external_volume_uom,
                b.water_based_adhesive,
                b.layer_category_1, 
                b.layer_category_2, 
                b.layer_category_3, 
                b.layer_category_4, 
                b.layer_category_5, 
                b.layer_category_6, 
                b.layer_category_7, 
                b.layer_category_8, 
                b.layer_category_9, 
                b.percentflexiblecomp1, 
                b.percentflexiblecomp2, 
                b.percentflexiblecomp3, 
                b.percentflexiblecomp4, 
                b.percentflexiblecomp5, 
                b.percentflexiblecomp6, 
                b.percentflexiblecomp7, 
                b.percentflexiblecomp8, 
                b.percentflexiblecomp9,
                b.layer_indicator,
                b.more_than_1_spec,
                b.matching_method,
                b.innerlinermaterial,
                CAST(ROUND(COALESCE(b.component_weight_g_override,b.total_weight) , 4) AS  DECIMAL(10, 4)) AS component_weight_g,
                b.override_flag,
                CONCAT_WS(',', 
                    IF(ARRAY_CONTAINS(b.rules_arr,'R1'), 
                        CONCAT('R1:(specnumber = ',             COALESCE(CAST(b.Source_specnumber AS STRING),''),       ' -> ', COALESCE(CAST(b.specnumber AS STRING),''), ')'), NULL),
                    IF(ARRAY_CONTAINS(b.rules_arr,'R2'),
                        CONCAT('R2:(component_weight_g = ',     COALESCE(CAST(ROUND(b.total_weight, 2) AS STRING),''),  ' -> ', COALESCE(CAST(ROUND(b.component_weight_g_override, 2) AS STRING),''), ')'), NULL),
                    IF(ARRAY_CONTAINS(b.rules_arr,'R3'),
                        CONCAT('R3:(component_usage = ',        COALESCE(CAST(b.component_usage AS STRING),''),         ' -> ', COALESCE(CAST(b.component_usage_override AS STRING),''), ')'), NULL),
                    IF(ARRAY_CONTAINS(b.rules_arr,'R4'),
                        CONCAT('R4:(component_category = ',     COALESCE(CAST(b.component_category AS STRING),''),      ' -> ', COALESCE(CAST(b.component_category_override AS STRING),''), ')'), NULL),
                    IF(ARRAY_CONTAINS(b.rules_arr,'R5'),
                        CONCAT('R5:(component_disposalcode = ', COALESCE(CAST(b.component_disposalcode AS STRING),''),  ' -> ', COALESCE(CAST(b.component_disposalcode_override AS STRING),''), ')'), NULL),
                    IF(ARRAY_CONTAINS(b.rules_arr,'R6'),
                        CONCAT('R6:(component_subcategory = ',  COALESCE(CAST(b.component_subcategory AS STRING),''),   ' -> ', COALESCE(CAST(b.component_subcategory_override AS STRING),''), ')'), NULL),
                    IF(ARRAY_CONTAINS(b.rules_arr,'R7'),
                        CONCAT('R7:(component_pack_group = ',   COALESCE(CAST(b.component_pack_group AS STRING),''),    ' -> ', COALESCE(CAST(b.component_pack_group_override AS STRING),''), ')'), NULL)
                ) AS override_rule
        FROM    base_ranked b
        WHERE   b.rn = 1  -- choose material/specnumber by component_weight_g + highest specnumber if multiple specs with same weight
    )

    SELECT      cte.*,
                COALESCE(cte.component_flex_rigid,g_flexrigid.LocalizedLabel)               AS component_flex_rigid_final,
                COALESCE(cte.component_usage,g_usage.LocalizedLabel)                        AS component_usage_final,
                COALESCE(cte.component_pack_group,g_packgroup.LocalizedLabel)               AS component_pack_group_final,
                COALESCE(cte.component_disposalcode,g_disposalcodegeneric.LocalizedLabel)   AS component_disposalcode_final,
                COALESCE(cte.component_is_main,g_main.LocalizedLabel)                       AS component_is_main_final,
                CASE WHEN cte.layer_category_1 IS NOT NULL THEN cte.layer_category_1
                        WHEN im.lk_categoryname IS NOT NULL
                            AND cte.basisweighttotal IS NOT NULL THEN im.lk_categoryname 
                END                                                                         AS layer_category_1_final, 
                CASE    WHEN cte.layer_category_2 IS NOT NULL THEN cte.layer_category_2 
                        WHEN im.lk_categoryname IS NOT NULL 
                            AND cte.basisweighttotal IS NOT NULL THEN "Paper" 
                END                                                                         AS layer_category_2_final,
                CASE    WHEN cte.percentflexiblecomp1 IS NOT NULL THEN cte.percentflexiblecomp1 
                        WHEN im.lk_categoryname IS NOT NULL 
                            AND cte.basisweighttotal IS NOT NULL THEN 
                                im.lk_weight/SUBSTRING(cte.basisweighttotal, 0,
                                LENGTH(cte.basisweighttotal)-5)
                END                                                                         AS percentflexiblecomp1_final,
                CASE    WHEN cte.percentflexiblecomp2 IS NOT NULL THEN cte.percentflexiblecomp2 
                        WHEN im.lk_categoryname IS NOT NULL 
                            AND cte.basisweighttotal IS NOT NULL THEN 
                                (SUBSTRING(cte.basisweighttotal, 0, 
                                LENGTH(cte.basisweighttotal)-5) - im.lk_weight) / 
                                SUBSTRING(cte.basisweighttotal, 0, 
                                LENGTH(cte.basisweighttotal)-5)
                END                                                                         AS percentflexiblecomp2_final,
                CASE    WHEN cte.layer_category_1 IS NOT NULL THEN cte.layer_indicator  
                        WHEN im.lk_categoryname IS NOT NULL
                            AND cte.basisweighttotal IS NOT NULL THEN "Carton"
                        ELSE cte.layer_indicator
                END                                                                         AS layer_indicator_final
    FROM        component cte
    LEFT JOIN   cr579_loraxpackfamilytopackgroup l
        ON      UPPER(cte.component_category)               = UPPER(l.cr579_packfamilyname)
        AND     UPPER(cte.component_subcategory)            = UPPER(l.cr579_packsubfamilyname)
    LEFT JOIN   GlobalOptionsetMetadata g_flexrigid 
        ON      UPPER(l.cr579_flexrigid)                    = UPPER(g_flexrigid.Option) 
        AND     UPPER(g_flexrigid.OptionSetName)            = UPPER('cr579_flexrigid')
    LEFT JOIN   GlobalOptionsetMetadata g_main 
        ON      UPPER(l.cr579_main)                         = UPPER(g_main.Option)
        AND     UPPER(g_main.OptionSetName)                 = UPPER('cr579_main')
    LEFT JOIN   GlobalOptionsetMetadata g_packgroup
        ON      UPPER(l.cr579_packgroup)                    = UPPER(g_packgroup.Option) 
        AND     UPPER(g_packgroup.OptionSetName)            = UPPER('cr579_packgroup')
    LEFT JOIN   GlobalOptionsetMetadata g_usage
        ON      UPPER(l.cr579_usage)                        = UPPER(g_usage.Option)
        AND     UPPER(g_usage.OptionSetName)                = UPPER('cr579_usage')
    LEFT JOIN   GlobalOptionsetMetadata g_disposalcodegeneric
        ON      UPPER(l.lk_disposalcodegeneric)             = UPPER(g_disposalcodegeneric.Option)
        AND     UPPER(g_disposalcodegeneric.OptionSetName)  = UPPER('lk_disposalcodegeneric')
    LEFT JOIN   cr579_pacinnermaterial im
        ON      TRIM(cte.innerlinermaterial) = TRIM(im.cr579_innerlinermaterial)
        AND     im.lk_weight IS NOT NULL
""").drop("layer_indicator") \
    .withColumns({
        "component_flex_rigid":     F.col("component_flex_rigid_final"),
        "component_usage":          F.col("component_usage_final"),
        "component_pack_group":     F.col("component_pack_group_final"),
        "component_disposalcode":   F.col("component_disposalcode_final"),
        "component_is_main":        F.col("component_is_main_final"),
        "layer_category_1":         F.col("layer_category_1_final"),
        "layer_category_2":         F.col("layer_category_2_final"),
        "percentflexiblecomp1":     F.col("percentflexiblecomp1_final"),
        "percentflexiblecomp2":     F.col("percentflexiblecomp2_final"),
        "Source":                   F.lit("ODS"),
        "SourceSystemUnit":         F.lit(core_source_system_id).cast("string"),
        "src_agnostic_unique_ID":   F.col("Material"),
        "_run_id":                  F.lit(job_id),
        "_run_timestamp" :          F.current_timestamp(),
        "specname":                 F.regexp_replace(F.regexp_replace("specname", r'\\\\+', r'\\'), r'\\"', '"')
    }) \
    .drop("component_flex_rigid_final", "component_usage_final", "component_pack_group_final", "component_disposalcode_final", "component_is_main_final", \
        "layer_category_1_final", "layer_category_2_final", "percentflexiblecomp1_final", "percentflexiblecomp2_final") \
    .withColumnRenamed("layer_indicator_final", "layer_indicator")

## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

## dim_component_hybrid - <u>manual/source agnostic</u>

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema":"output", "target_table":"dim_component_hybrid", "format":"delta","opertation_type":"Managed Table Type - Append"},
    "source_tables": [
        {"schema":"output",     "table":"dim_component_hybrid",                 "view":"dim_component_hybrid"},
        {                       "table":"dbo.lk_component",                     "view":"lk_component",                      "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"},
        {                       "table":"dbo.GlobalOptionsetMetadata",          "view":"GlobalOptionsetMetadata",           "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"},
        {                       "table":"dbo.cr579_loraxpackfamilytopackgroup", "view":"cr579_loraxpackfamilytopackgroup",  "linked_service_name":"DATAVERSE_SL_SQL_LS",    "opertation_type":"serverless jdbc ls"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

from pyspark.sql import types as T, functions as F

# Get data from Manual Components PowerApp table ...
df = spark.sql("""
    SELECT      COALESCE(lk_localcomponentid, lk_id)                            AS material,
                lk_description                                                  AS component_description,
                'VERP'                                                          AS material_type,
                'Active'                                                        AS material_status,
                lk_categoryname                                                 AS component_category,
                lk_subcategoryname                                              AS component_subcategory,
                lk_weight                                                       AS component_weight_g,
                lk_height                                                       AS component_height,
                lk_width                                                        AS component_width,
                lk_depth                                                        AS component_depth,
                cr579_dimensionuom                                              AS component_dimension_uom,
                lk_depth * lk_width                                             AS component_area,
                lk_depth * lk_width * lk_height                                 AS component_volume,
                COALESCE(lk_usagename, gm_usage.LocalizedLabel)                 AS component_usage,
                COALESCE(lk_packgroupname, gm_pack_group.LocalizedLabel)        AS component_pack_group,
                COALESCE(lk_disposalcodename, gm_disposalcode.LocalizedLabel)   AS component_disposalcode,
                CASE WHEN lk_isrigid = 'True'   THEN 'Rigid'
                     WHEN lk_isrigid = 'False'  THEN 'Flexible'
                     ELSE gm_flex_rigid.LocalizedLabel
                END                                                             AS component_flex_rigid,
                gm_main.LocalizedLabel                                          AS component_is_main,
                    CONCAT(CASE WHEN lk_sourcesystemunit = 1 THEN '' ELSE CONCAT(lk_sourcesystemunit,':') END, COALESCE(lk_localcomponentid, lk_id), '-001')
                                                                                AS specnumber,
                CAST(lk_sourcesystemunit AS STRING)                             AS SourceSystemUnit,
                    CONCAT(CASE WHEN lk_sourcesystemunit = 1 THEN '' ELSE CONCAT(lk_sourcesystemunit,':') END, COALESCE(lk_localcomponentid, lk_id))
                                                                                AS src_agnostic_unique_ID
    FROM        lk_component comp
    LEFT JOIN   cr579_loraxpackfamilytopackgroup l
        ON      comp.lk_categoryname = l.cr579_packfamilyname
        AND     comp.lk_subcategoryname = l.cr579_packsubfamilyname
    LEFT JOIN   GlobalOptionsetMetadata gm_flex_rigid
        ON      l.cr579_flexrigid = gm_flex_rigid.Option
        AND     gm_flex_rigid.OptionSetName = 'cr579_flexrigid'
    LEFT JOIN   GlobalOptionsetMetadata gm_pack_group
        ON      l.cr579_packgroup = gm_pack_group.Option
        AND     gm_pack_group.OptionSetName = 'cr579_packgroup'
    LEFT JOIN   GlobalOptionsetMetadata gm_usage
        ON      l.cr579_usage = gm_usage.Option
        AND     gm_usage.OptionSetName = 'cr579_usage'
    LEFT JOIN   GlobalOptionsetMetadata gm_disposalcode
        ON      l.lk_disposalcodegeneric = gm_disposalcode.Option
        AND     gm_disposalcode.OptionSetName = 'lk_disposalcodegeneric'
    LEFT JOIN   GlobalOptionsetMetadata gm_main
        ON      l.cr579_main = gm_main.Option
        AND     gm_main.OptionSetName = 'cr579_main' 
""").createOrReplaceTempView("df_manual")

# Filter out any components already found in dim_component_hybrid (shouldn't be the case / possible anyway, but better safe than sorry), align table schema.
df=spark.sql("""
    SELECT      man.*
    FROM        df_manual man
    ANTI JOIN   dim_component_hybrid ch
        ON      ch.src_agnostic_unique_id = man.src_agnostic_unique_id
""").withColumns({
        "component_area":           F.col("component_area").cast(T.DoubleType()),
        "component_depth":          F.col("component_depth").cast(T.StringType()),
        "component_width":          F.col("component_width").cast(T.StringType()),
        "component_height":         F.col("component_height").cast(T.StringType()),
        "component_weight_g":       F.col("component_weight_g").cast(T.DecimalType(10,4)),
        "component_volume":         F.col("component_volume").cast(T.DoubleType()),
        "component_dimension_uom":  F.col("component_dimension_uom").cast(T.StringType()),
        "Source":                   F.lit("Manual"),
        "_run_id":                  F.lit(job_id),
        "_run_timestamp" :          F.current_timestamp()
    }).cache()

print("Appending", df.count(), "manual components ...")

## ------------------------------------------------------------------------------------
create_table(metadata_json, df)
df.unpersist();

## dim_component_hybrid - orphaned components referred to in BOM Versions

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema":"output", "target_table":"dim_component_hybrid", "format":"delta","opertation_type":"Managed Table Type - Append"},
    "source_tables": [
        {"schema":"output",     "table":"dim_component_hybrid",                         "view":"dim_component_hybrid"},
        {"schema":"output",     "table":"lorax_dim_bill_of_material_persist_detail",    "view":"lorax_dim_bill_of_material_persist_detail", "project_name":"lorax", "params":{"optional":"True"}},
        {"schema":"output",     "table":"lorax_dim_component_hybrid",                   "view":"lorax_dim_component_hybrid",                "project_name":"lorax", "params":{"optional":"True"}}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

from pyspark.sql import types as T, functions as F

# Find components that are used in BOM Versions but no longer present in dim_component_hybrid (this can happen for Manual Components and Veritas 'unmatched spec' components)
# Copy the data back over from lorax_output.lorax_dim_component_hybrid (which is a 'history' table so it has deleted records too).
# The fact of copying them back in to dim_component_hybrid will then change the operation_type from D to U in lorax_output.lorax_dim_component_hybrid when the lorax_process/
#    lorax_output update runs.

# If either of the 2 lorax_output tables doesn't exist, skip (next pipeline run will then pick up again, as these tables should be there after a complete end-to-end run)
chk_tbl_1 = delta_table_exists("lorax_dim_component_hybrid")
chk_tbl_2 = delta_table_exists("lorax_dim_bill_of_material_persist_detail")

if chk_tbl_1 and chk_tbl_2:

    df = spark.sql("""
        SELECT      *
        FROM        lorax_dim_component_hybrid lx_comp
        SEMI JOIN   lorax_dim_bill_of_material_persist_detail bom
            ON      lx_comp.src_agnostic_unique_id = bom.component
        ANTI JOIN   dim_component_hybrid comp
            ON      comp.src_agnostic_unique_id = lx_comp.src_agnostic_unique_id
    """) \
        .withColumns({
            "Source":           F.lit("BOM Version orphaned components"),
            "_run_id":          F.lit(job_id),
            "_run_timestamp" :  F.current_timestamp()
        }) \
        .drop("operation_type", "last_update_dt") \
        .cache()

    print("Appending", df.count(), "orphaned components (present in BOM Versions but no longer in dim_component_hybrid) ...")

    ## ------------------------------------------------------------------------------------
    create_table(metadata_json, df)
    df.unpersist();

else:
    print("Skipping orphaned components: required tables missing.")

## fact_helios_sales_skuprint

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema":"output", "target_table":"fact_helios_sales_skuprint", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"schema":"raw"     , "table": "helios_sold_goods"                      , "view": "helios_sold_goods"},
        {"schema":"raw"     , "table": "helios_unit_hierarchy"                  , "view": "helios_unit_hierarchy"},
        {"schema":"raw"     , "table": "helios_unit"                            , "view": "helios_unit"},
        {"schema":"output"  , "table": "dim_material"                           , "view": "dim_material"},
        {"table":"dbo.v_odsrules", "view":"v_odsrules", "linked_service_name":"DATAVERSE_SL_SQL_LS", "opertation_type":"serverless jdbc ls"},
        {"table":"dbo.cr579_countries_to_subsegments","view":"cr579_countries_to_subsegments",  "linked_service_name":"DATAVERSE_SL_SQL_LS", "opertation_type":"serverless jdbc ls"},
        {"table":"dbo.GlobalOptionsetMetadata", "view":"GlobalOptionsetMetadata",   "linked_service_name":"DATAVERSE_SL_SQL_LS", "opertation_type":"serverless jdbc ls"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

# ======================================================================================================================================================
# =====================                              1. Prepare helios_unit_mapping_skuprint_unpivot                               =====================
# ======================================================================================================================================================

from pyspark.sql.functions import array, col, explode, lit, arrays_zip

SubSegment_to_Division_df = spark.sql("""
    SELECT      Value1 as SubSegment_Code,
                Value2 as SubSegment_Name,
                Value3 as Mars_Division
    FROM        v_odsrules
    WHERE       Function = 'HELIOS_UNIT_MAPPING_SKUPRINT'
    AND         Criteria = 'MARS_DIVISION'
""")

#Read table
df = spark.table("cr579_countries_to_subsegments")

#Metadata table fo EntityName = "cr579_countries_to_subsegments"
GlobalOptionsetMetadata_df = spark.sql("""SELECT * FROM GlobalOptionsetMetadata WHERE EntityName = "cr579_countries_to_subsegments" """)

# Get all SubSegment columns based on name
columns_to_unpivot = [c for c in df.columns if c.lower().endswith("subsegment") ]
columns_to_unpivot = [c for c in columns_to_unpivot if c.lower().startswith("cr579") ]

print(f"Columns to unpivot: {columns_to_unpivot}")

#unpivot helios_unit_mapping_skuprint by columns_to_unpivot
df_unpivot = df.withColumn(
    "Unpivoted",
    explode(
        arrays_zip(
            array([lit(c) for c in columns_to_unpivot]).alias("SubSegment_Code"),
            array([col(c) for c in columns_to_unpivot]).alias("SubSegment_ID")
        )
    )
).select(
    "cr579_country", 
    "cr579_country_code", 
    "cr579_country_code_helios",
    col("Unpivoted.SubSegment_Code").alias("SubSegment_Code"),
    col("Unpivoted.SubSegment_ID").alias("SubSegment_ID")
).distinct()


#Get SubSegment labels from GlobalOptionsetMetadata table
df_unpivot = df_unpivot.alias('um').join(GlobalOptionsetMetadata_df.alias("g"), (F.col("um.SubSegment_Code") == F.col("g.OptionSetName")) & (F.col("um.SubSegment_ID") == F.col("g.Option")), how = 'left')\
                    .selectExpr("um.*","g.LocalizedLabel AS SubSegment")

#Join to SubSegment to Mars_Division mapping
df_unpivot = df_unpivot.alias('um').join(SubSegment_to_Division_df.alias("s"), F.col("um.SubSegment_Code") == F.col("s.SubSegment_Code"), how = 'left')\
                    .select("um.*","s.Mars_Division", "s.SubSegment_Name")

df_unpivot.createOrReplaceTempView("helios_unit_mapping_skuprint_unpivot")

# ======================================================================================================================================================
# =====================                                                 2. Prepare output                                          =====================
# ======================================================================================================================================================

case_statement_Segment_Final = get_packrule_by_columns("v_odsrules", "HELIOS_MARS_SEGMENT_AND_DIVISION", "FACT_SALES", ['KEY1','KEY2'], ['VALUE1','VALUE2'], ['FN1', 'FN2'], 'RESULT1')
print(case_statement_Segment_Final)

case_statement_Segment_Division = get_packrule_by_columns("v_odsrules", "HELIOS_MARS_SEGMENT_AND_DIVISION", "FACT_SALES", ['KEY1','KEY2'], ['VALUE1','VALUE2'], ['FN1', 'FN2'], 'RESULT2')
print(case_statement_Segment_Division)

join_statement_SubSegment_Division = get_packrule_by_columns("v_odsrules", "HELIOS_MARS_SEGMENT_AND_DIVISION", "FACT_SALES", ['KEY1','KEY2'], ['VALUE1','VALUE2'], ['FN1', 'FN2'], 'RESULT3')
print(join_statement_SubSegment_Division)

### OE (Operation_Entity) Mapping
case_statement_OE_Field_Level4 = get_packrule_by_columns("v_odsrules", "OE_FIELD_MAPPING", "BY_COLUMN", ['KEY1'], ['VALUE1'], ['FN1'], 'RESULT1')
case_statement_OE_Field_Level5 = get_packrule_by_columns("v_odsrules", "OE_FIELD_MAPPING", "BY_COLUMN", ['KEY2'], ['VALUE1'], ['FN2'], 'RESULT1')
case_statement_OE_Field_Level6 = get_packrule_by_columns("v_odsrules", "OE_FIELD_MAPPING", "BY_COLUMN", ['KEY3'], ['VALUE1'], ['FN3'], 'RESULT1')
case_statement_OE_Field_Level4_Unit = get_packrule_by_columns("v_odsrules", "OE_FIELD_MAPPING", "BY_COLUMN_NAME", ['KEY4'], ['VALUE1'], ['FN4'], 'RESULT1')

df = spark.sql(f"""
    WITH helios_unit_table as (
        SELECT      Unit_ID,
                    Country_Code,
                    Country_Name,
                    Description
        FROM        helios_unit
    ),
    hierarchy as (
        SELECT      coalesce(h.Level12_Unit_ID,h.Level11_Unit_ID,h.Level10_Unit_ID,h.Level9_Unit_ID,h.Level8_Unit_ID,h.Level7_Unit_ID,h.Level6_Unit_ID,h.Level5_Unit_ID,h.Level4_Unit_ID,h.Level3_Unit_ID,h.Level2_Unit_ID,h.Level1_Unit_ID) as Unit_ID,
                    CASE    WHEN h.Level12_Unit_ID IS NOT NULL THEN 12
                            WHEN h.Level11_Unit_ID IS NOT NULL THEN 11
                            WHEN h.Level10_Unit_ID IS NOT NULL THEN 10
                            WHEN h.Level9_Unit_ID IS NOT NULL THEN 9
                            WHEN h.Level8_Unit_ID IS NOT NULL THEN 8
                            WHEN h.Level7_Unit_ID IS NOT NULL THEN 7
                            WHEN h.Level6_Unit_ID IS NOT NULL THEN 6
                            WHEN h.Level5_Unit_ID IS NOT NULL THEN 5
                            WHEN h.Level4_Unit_ID IS NOT NULL THEN 4
                            WHEN h.Level3_Unit_ID IS NOT NULL THEN 3
                            WHEN h.Level2_Unit_ID IS NOT NULL THEN 2
                            WHEN h.Level1_Unit_ID IS NOT NULL THEN 1
                            ELSE 0 
                    END as Unit_Level,
                    h.Level12_Unit_ID,
                    u12.Description as Level12_Description,
                    concat(h.Level12_Unit_ID,'_', u12.Description) as Full_Level12_Description,
                    h.Level11_Unit_ID,
                    u11.Description as Level11_Description,
                    concat(h.Level11_Unit_ID,'_', u11.Description) as Full_Level11_Description,
                    h.Level10_Unit_ID,
                    u10.Description as Level10_Description,
                    concat(h.Level10_Unit_ID,'_', u10.Description) as Full_Level10_Description,
                    h.Level9_Unit_ID,
                    u9.Description as Level9_Description,
                    concat(h.Level9_Unit_ID,'_', u9.Description) as Full_Level9_Description,
                    h.Level8_Unit_ID,
                    u8.Description as Level8_Description,
                    concat(h.Level8_Unit_ID,'_', u8.Description) as Full_Level8_Description,
                    h.Level7_Unit_ID,
                    u7.Description as Level7_Description,
                    concat(h.Level7_Unit_ID,'_', u7.Description) as Full_Level7_Description,
                    h.Level6_Unit_ID,
                    u6.Description as Level6_Description,
                    concat(h.Level6_Unit_ID,'_', u6.Description) as Full_Level6_Description,
                    h.Level5_Unit_ID,
                    u5.Description as Level5_Description,
                    concat(h.Level5_Unit_ID,'_', u5.Description) as Full_Level5_Description,
                    h.Level4_Unit_ID,
                    u4.Description as Level4_Description,
                    concat(h.Level4_Unit_ID,'_', u4.Description) as Full_Level4_Description,
                    h.Level3_Unit_ID,
                    u3.Description as Level3_Description,
                    concat(h.Level3_Unit_ID,'_', u3.Description) as Full_Level3_Description,
                    h.Level2_Unit_ID,
                    u2.Description as Level2_Description,
                    concat(h.Level2_Unit_ID,'_', u2.Description) as Full_Level2_Description,
                    h.Level1_Unit_ID,
                    u1.Description as Level1_Description,
                    concat(h.Level1_Unit_ID,'_', u1.Description) as Full_Level1_Description
        FROM        helios_unit_hierarchy h
        LEFT JOIN   helios_unit_table u1
            ON      h.Level1_Unit_ID = u1.Unit_ID
        LEFT JOIN   helios_unit_table u2
            ON      h.Level2_Unit_ID = u2.Unit_ID
        LEFT JOIN   helios_unit_table u3
            ON      h.Level3_Unit_ID = u3.Unit_ID
        LEFT JOIN   helios_unit_table u4
            ON      h.Level4_Unit_ID = u4.Unit_ID
        LEFT JOIN   helios_unit_table u5
            ON      h.Level5_Unit_ID = u5.Unit_ID
        LEFT JOIN   helios_unit_table u6
            ON      h.Level6_Unit_ID = u6.Unit_ID
        LEFT JOIN   helios_unit_table u7
            ON      h.Level7_Unit_ID = u7.Unit_ID
        LEFT JOIN   helios_unit_table u8
            ON      h.Level8_Unit_ID = u8.Unit_ID
        LEFT JOIN   helios_unit_table u9
            ON      h.Level9_Unit_ID = u9.Unit_ID
        LEFT JOIN   helios_unit_table u10
            ON      h.Level10_Unit_ID = u10.Unit_ID
        LEFT JOIN   helios_unit_table u11
            ON      h.Level11_Unit_ID = u11.Unit_ID
        LEFT JOIN   helios_unit_table u12
            ON      h.Level12_Unit_ID = u12.Unit_ID
        WHERE       h.Level1_Unit_ID = "12000" 

        UNION 
        
        SELECT      Unit_ID,
                    1 as Unit_Level,
                    null as Level12_Unit_ID,
                    null as Level12_Description,
                    null as Full_Level12_Description,
                    null as Level11_Unit_ID,
                    null as Level11_Description,
                    null as Full_Level11_Description,
                    null as Level10_Unit_ID,
                    null as Level10_Description,
                    null as Full_Level10_Description,
                    null as Level9_Unit_ID,
                    null as Level9_Description,
                    null as Full_Level9_Description,
                    null as Level8_Unit_ID,
                    null as Level8_Description,
                    null as Full_Level8_Description,
                    null as Level7_Unit_ID,
                    null as Level7_Description,
                    null as Full_Level7_Description,
                    null as Level6_Unit_ID,
                    null as Level6_Description,
                    null as Full_Level6_Description,
                    null as Level5_Unit_ID,
                    null as Level5_Description,
                    null as  Full_Level5_Description,
                    null as Level4_Unit_ID,
                    null as  Level4_Description,
                    null as  Full_Level4_Description,
                    null as Level3_Unit_ID,
                    null as Level3_Description,
                    null as Full_Level3_Description,
                    null as Level2_Unit_ID,
                    null as Level2_Description,
                    null as Full_Level2_Description,
                    Unit_ID as Level1_Unit_ID,
                    h.Description as Level1_Description,
                    concat(h.Unit_ID,'_', h.Description) as Full_Level1_Description
        FROM        helios_unit h
        WHERE       Unit_ID = "12000" 
    ), 
    sold_goods AS (
        SELECT      sd.Common_Item_ID,
                    sd.Unit_ID,
                    sd.Unit,
                    sd.Market_ID,
                    sd.Market,
                    sd.Level4_Unit,
                    sd.Level5_Unit,
                    sd.Level6_Unit,
                    sd.Period,
                    sd.Year,
                    sum(sd.Tonnes) as Tonnes,
                    max(NSV_Cat) as NSV_Cat,
                    sum(sd.GSV) as GSV
        FROM        helios_sold_goods sd
        GROUP BY    sd.Common_Item_ID,
                    sd.Unit_ID,
                    sd.Unit,
                    sd.Market_ID,
                    sd.Market,
                    sd.Level4_Unit,
                    sd.Level5_Unit,
                    sd.Level6_Unit,
                    sd.Period,
                    sd.Year
    ),
    main_query AS (
        SELECT      sd.Common_Item_ID,
                    sd.Unit_ID,
                    sd.Unit,
                    sd.Market_ID,
                    sd.Market,
                    u.Country_Code,
                    u.Country_Name,
                    sd.Level4_Unit,
                    sd.Level5_Unit,
                    sd.Level6_Unit,
                    sd.Period,
                    sd.Year,
                    sd.Tonnes,
                    sd.NSV_Cat,
                    sd.GSV,
                    CASE    WHEN sd.Tonnes > 0 THEN 'Positive Tonnes'
                            WHEN sd.Tonnes < 0 THEN 'Negative Tonnes'
                            WHEN sd.Tonnes = 0 THEN 'Zero Tonnes'
                            ELSE ''
                    END as Tonnes_Category,
                    m.Mars_Segment,
                    m.Mars_Division,
                    h.Unit_Level,
                    h.Level1_Unit_ID,
                    h.Level1_Description,
                    h.Full_Level1_Description,
                    h.Level2_Unit_ID,
                    h.Level2_Description,
                    h.Full_Level2_Description,
                    h.Level3_Unit_ID,
                    h.Level3_Description,
                    h.Full_Level3_Description,
                    h.Level4_Unit_ID,
                    h.Level4_Description,
                    h.Full_Level4_Description,
                    h.Level5_Unit_ID,
                    h.Level5_Description,
                    h.Full_Level5_Description,
                    h.Level6_Unit_ID,
                    h.Level6_Description,
                    h.Full_Level6_Description,
                    h.Level7_Unit_ID,
                    h.Level7_Description,
                    h.Full_Level7_Description,
                    h.Level8_Unit_ID,
                    h.Level8_Description,
                    h.Full_Level8_Description,
                    h.Level9_Unit_ID,
                    h.Level9_Description,
                    h.Full_Level9_Description,
                    h.Level10_Unit_ID,
                    h.Level10_Description,
                    h.Full_Level10_Description,
                    h.Level11_Unit_ID,
                    h.Level11_Description,
                    h.Full_Level11_Description,
                    h.Level12_Unit_ID,
                    h.Level12_Description,
                    h.Full_Level12_Description
        FROM        sold_goods sd
        LEFT JOIN   hierarchy h
            ON      sd.Unit_ID = h.Unit_ID
        LEFT JOIN   helios_unit_table u
            ON      sd.Unit_ID = u.Unit_ID
        LEFT JOIN   dim_material m
            ON      m.Material = sd.Common_Item_ID
    ), 
    main_query_with_rules AS (
        SELECT      Common_Item_ID,
                    Unit_ID,
                    Unit,
                    Market_ID,
                    Market,
                    Country_Code,
                    Country_Name,
                    Level4_Unit,
                    Level5_Unit,
                    Level6_Unit,
                    Period,
                    Year,
                    Tonnes,
                    NSV_Cat,
                    GSV,
                    Tonnes_Category,
                    coalesce({case_statement_Segment_Final},"Unknown") as Business_Segment,
                    coalesce({case_statement_Segment_Division},"Unknown") as Business_Division,
                    coalesce({join_statement_SubSegment_Division},"Unknown") as join_Business_Division,
                    Mars_Segment,
                    Mars_Division,
                    Unit_Level,
                    Level1_Unit_ID,
                    Level1_Description,
                    Full_Level1_Description,
                    Level2_Unit_ID,
                    Level2_Description,
                    Full_Level2_Description,
                    Level3_Unit_ID,
                    Level3_Description,
                    Full_Level3_Description,
                    Level4_Unit_ID,
                    Level4_Description,
                    Full_Level4_Description,
                    Level5_Unit_ID,
                    Level5_Description,
                    Full_Level5_Description,
                    Level6_Unit_ID,
                    Level6_Description,
                    Full_Level6_Description,
                    Level7_Unit_ID,
                    Level7_Description,
                    Full_Level7_Description,
                    Level8_Unit_ID,
                    Level8_Description,
                    Full_Level8_Description,
                    Level9_Unit_ID,
                    Level9_Description,
                    Full_Level9_Description,
                    Level10_Unit_ID,
                    Level10_Description,
                    Full_Level10_Description,
                    Level11_Unit_ID,
                    Level11_Description,
                    Full_Level11_Description,
                    Level12_Unit_ID,
                    Level12_Description,
                    Full_Level12_Description,
                    CASE {case_statement_OE_Field_Level4[5:-20]} {case_statement_OE_Field_Level5[5:-20]} {case_statement_OE_Field_Level6[5:-20]} {case_statement_OE_Field_Level4_Unit[5:-20]}  END as Operation_Entity
        FROM        main_query
    )
    SELECT      q.Common_Item_ID,
                q.Unit_ID,
                q.Unit,
                q.Market_ID,
                q.Market,
                q.Country_Code,
                q.Country_Name,
                q.Level4_Unit,
                q.Level5_Unit,
                q.Level6_Unit,
                q.Period,
                q.Year,
                q.Tonnes,
                q.NSV_Cat,
                q.GSV,
                q.Tonnes_Category,
                q.Business_Segment,
                q.Business_Division,
                q.Mars_Segment,
                q.Mars_Division,
                um.SubSegment_Name,
                um.SubSegment,
                q.Unit_Level,
                q.Level1_Unit_ID,
                q.Level1_Description,
                q.Full_Level1_Description,
                q.Level2_Unit_ID,
                q.Level2_Description,
                q.Full_Level2_Description,
                q.Level3_Unit_ID,
                q.Level3_Description,
                q.Full_Level3_Description,
                q.Level4_Unit_ID,
                q.Level4_Description,
                q.Full_Level4_Description,
                q.Level5_Unit_ID,
                q.Level5_Description,
                q.Full_Level5_Description,
                q.Level6_Unit_ID,
                q.Level6_Description,
                q.Full_Level6_Description,
                q.Level7_Unit_ID,
                q.Level7_Description,
                q.Full_Level7_Description,
                q.Level8_Unit_ID,
                q.Level8_Description,
                q.Full_Level8_Description,
                q.Level9_Unit_ID,
                q.Level9_Description,
                q.Full_Level9_Description,
                q.Level10_Unit_ID,
                q.Level10_Description,
                q.Full_Level10_Description,
                q.Level11_Unit_ID,
                q.Level11_Description,
                q.Full_Level11_Description,
                q.Level12_Unit_ID,
                q.Level12_Description,
                q.Full_Level12_Description,
                q.Operation_Entity
    FROM        main_query_with_rules q
    LEFT JOIN   helios_unit_mapping_skuprint_unpivot um
        ON      q.Country_Code = um.cr579_country_code_helios
        AND     um.Mars_Division = q.join_Business_Division
""").withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df)

<mark>**USAGE STATISTICS - FACT ACTIVITY**</mark>

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "usr_stats_fact_activity", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"schema":"raw",        "table":"user_stats_power_bi",          "view":"df_act_pbi"},
        {"schema":"raw",        "table":"user_stats_pbi_user_list",     "view":"df_act_pbi_user_list"},
        {"schema":"raw",        "table":"user_stats_ms_clarity",        "view":"df_act_web"},
        {"schema":"process",    "table":"usr_stats_dim_user",           "view":"df_usr"},
        {"view_name":"df_act_pbi_to_include", "path":"/USER_STATS/activity_to_include.csv", "file_format":"csv", "container":"files" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service", "csv_options":{ "inferSchema": "true", "header": "true"}},            
        {"view_name":"df_act_gpt", "path":"/USER_STATS/PACKGPT/", "file_format":"csv", "container":"files" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service", "csv_options":{ "inferSchema": "true", "header": "true"}},
        {"view_name":"df_act_pwa", "path":"/USER_STATS/POWERAPP_REPORTS/", "file_format":"csv", "container":"files" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service", "csv_options":{ "inferSchema": "true", "header": "true"}},
        {"view_name":"df_wrk", "path":"/USER_STATS/workspaces.csv", "file_format":"csv", "container":"files" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service", "csv_options":{ "inferSchema": "true", "header": "true"}}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

from pyspark.sql.functions import split
from pyspark.sql.functions import regexp_replace, trim, col

#---------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
# FACT TABLE
# CONCATENATES DATA FROM MULTIPLE SOURCES:
# 1.PBI ACTIVITY REPORT FROM PBI SUPPORT TEAM
# 2.PBI USER LIST FROM PBI API
# 3.PackGTP USAGE STATS FROM MANUAL EXTRACT
# 4.MS CLARITY DATA FOR WEBAPP USAGE STATS
# 5.POWERAPP USER ACTIVITY FROM MANUAL EXTRACT
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------------------------------------------------

df_act_p = spark.sql("""
    WITH df_main_pbi AS (
        SELECT      a.ID as id,
                    md5(lower(a.UserID)) as usr_id,
                    a.UserID as usr_email,
                    a.WorkspaceID as workspace_id,
                    a.AppID as app_id,
                    a.Activity as activity,
                    a.DistributionMethod as distribution_method,
                    a.DatasetName as dataset_name,
                    a.ItemName as item_name,
                    a.ReportName as report_name,
                    DATE_FORMAT(TO_DATE(a.CreationDate, 'M/d/yyyy'), 'yyyy-MM-dd') as creation_date,
                    a.CreationTime as creation_time,
                    '1' as usr_cnt
        FROM        df_act_pbi a
        JOIN        df_act_pbi_to_include b
            ON      a.activity = b.activity
            AND     b.include = 'Y'
    ),
    df_usr_trunc AS (
        SELECT      LEFT(usr_email,15) as usr_email_trunc,
                    usr_email
        FROM        df_usr
    ),
    df_main_gpt_0 AS (
        SELECT      coalesce(b.usr_email,a.user_id) as usr_email,
                    a.message_creation_timestamp as creation_timestamp,
                    TO_DATE(LEFT(a.message_creation_timestamp,10), 'MM/dd/yyyy') as creation_date,
                    DATE_FORMAT(a.message_creation_timestamp, 'hh:mm:ss') as creation_time,
                    HOUR(DATE_FORMAT(a.message_creation_timestamp, 'HH:mm:ss')) as creation_hour,
                    a.*
        FROM        df_act_gpt a
        LEFT JOIN   df_usr_trunc b
            ON      a.user_id = b.usr_email_trunc
        WHERE       a.user_id NOT IN ('test_user')
            AND     a.message_type = ('session_start')
        ORDER BY    a.message_creation_timestamp DESC
    ),
    df_main_gpt_1 AS (
        SELECT      usr_email,
                    creation_date,
                    creation_hour,
                    message_type,
                    MIN(session_id) as session_id
        FROM        df_main_gpt_0
        GROUP BY    usr_email,
                    creation_date,
                    creation_hour,
                    message_type
        ORDER BY    creation_date DESC
    ),
    df_main_gpt_2 AS (
        SELECT  session_id as ID,
                md5(lower(usr_email)) as usr_id,
                usr_email,
                'packgptdevsqls.database.windows.net' as workspace_id,
                'PackGTP' as app_id,
                --message_type  as activity,
                'SessionStart' as activity,
                'GenAI' as distribution_method,
                'PackGPT' as dataset_name,
                'PackGPT' as item_name,            
                'PackGPT' as report_name,
                creation_date,
                CONCAT(creation_hour,':00:00') as creation_time,
                '1' as usr_cnt
        FROM    df_main_gpt_1
    ),
    df_act_pbi_user_list_0 AS (
        SELECT  hashkey as ID,
                md5(lower(emailAddress)) as usr_id,
                lower(emailAddress) as usr_email,
                workspace_id,
                app_id,
                'NewUserEvent' as activity,
                'Apps' as distribution_method,
                'Not Applicable' as dataset_name,
                'Not Applicable' as item_name,
                'Not Applicable' as report_name,
                DATE_FORMAT(timestamp, 'yyyy-MM-dd') as creation_date,
                DATE_FORMAT(timestamp, 'hh:mm:ss') as creation_time,
                '1' as usr_cnt
        FROM    df_act_pbi_user_list
        WHERE   userType IS NOT NULL
            AND Deleted = False
    ),
    df_pwa_0 AS (
        SELECT  DISTINCT     
                `App name` AS app_name,
                `'Environments'[environmentId]` as env_id,
                `Unique users` as unique_users,
                `tenantId` as tenant_id,
                `App id` as app_id,
                `subType` as sub_type,
                `Platform` as platform,
                `Time accessed` as time_accessed,
                `tenantIdplusenvIdplusappId` as tenant_env_app_id
        FROM    df_act_pwa
    ),
    df_pwa_1 AS (
        SELECT  md5(concat(a.tenant_env_app_id,a.time_accessed)) as ID,
                'Unknown' AS usr_id,
                'Unknown' AS usr_email,
                concat(a.env_id,'|',a.app_id) as workspace_id,
                a.app_id,
                'ViewReport' as activity,
                'PowerApps' as distribution_method,
                'Not Applicable' as dataset_name,
                'Not Applicable' as item_name,
                'Not Applicable' as report_name,
                TO_DATE(replace(a.time_accessed,' 0:00',''), 'M/d/yyyy') as creation_date,
                '00:00:00' as creation_time,
                a.unique_users as usr_cnt
        FROM    df_pwa_0 a
        JOIN    df_wrk b
            ON  concat(a.env_id,'|',a.app_id) = b.workspace_id
    ),
    df_web_0 AS (
        SELECT  hashkey as ID,
                md5(upper(Country)) as usr_id,
                lower(Country) as usr_email,
                concat(workspace_name,'|',workspace_tier) as workspace_id,
                concat(workspace_name,'|',workspace_tier) as app_id,
                'SessionCount' as activity,
                'WebApps' as distribution_method,
                'Not Applicable' as dataset_name,
                'Not Applicable' as item_name,
                'Not Applicable' as report_name,
                DATE_FORMAT(IngestDate, 'yyyy-MM-dd') as creation_date,
                DATE_FORMAT(IngestDate, 'hh:mm:ss') as creation_time,
                totalSessionCount as usr_cnt
        FROM    df_act_web
    ),
    df_final AS (
                SELECT * FROM df_main_pbi WHERE creation_date >= '2023-01-01'   -- PBI
        UNION   SELECT * FROM df_main_gpt_2                                     -- GPT
        UNION   SELECT * FROM df_act_pbi_user_list_0                            -- PBI USER LIST
        UNION   SELECT * FROM df_pwa_1                                          -- POWER APP
        UNION   SELECT * FROM df_web_0                                          -- WEB APP
    )

    SELECT * FROM df_final
""").withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df_act_p)

<mark>**USAGE STATISTICS - DIM CALENDAR**</mark>

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "usr_stats_dim_calendar", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"schema":"output",     "table":"dim_calendar_day",                                "view":"df_cal"},
        {"schema":"output",     "table":"usr_stats_fact_activity",                         "view":"df_act"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

from pyspark.sql.functions import *

df_cal_o = spark.sql("""
    WITH df_day_list AS (
        SELECT      DISTINCT creation_date
        FROM        df_act a
        ORDER BY    creation_date     
    ),
    df_cal_final AS (
        SELECT      a.* 
        FROM        df_cal a
        JOIN        df_day_list b
        ON          a.day = b.creation_date
    )

    SELECT      day,
                year,
                period,
                quarter,
                week
    FROM        df_cal_final 
    ORDER BY    day
""").withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df_cal_o)

<mark>**USAGE STATISTICS - DIM WORKSPACE**</mark>

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "usr_stats_dim_workspace", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"view_name":"df_wrk", "path":"/USER_STATS/workspaces.csv", "file_format":"csv", "container":"files" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service", "csv_options":{ "inferSchema": "true", "header": "true"}}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

df_wrk_o = spark.sql("""
    SELECT      *
    FROM        df_wrk
""").withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df_wrk_o)

<mark>**USAGE STATISTICS - DIM USER**</mark>

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "usr_stats_dim_user", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"schema":"process",        "table":"usr_stats_dim_user",               "view":"df_usr"},
        {"schema":"output",         "table":"usr_stats_fact_activity",          "view":"df_act"},
        {"schema":"raw",            "table":"units_guide_units_details",        "view":"df_units"},
        {"schema":"raw",            "table":"user_stats_ms_clarity",            "view":"df_act_web"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

df_usr_o = spark.sql("""
    WITH df_email_act AS (
        SELECT DISTINCT     a.usr_id,
                            a.usr_email
        FROM                df_act a 
        WHERE               distribution_method not in ('WebApps')    
    ),
    df_user_list AS (
        SELECT              a.*
        FROM                df_email_act a
        WHERE NOT EXISTS    (   SELECT 1 
                                FROM df_usr b 
                                WHERE a.usr_id = b.usr_id
                            )
    ),
    df_user_missing AS (
        SELECT          usr_id,
                        usr_email,
                        'Unknown' as usr_type,
                        'WNL' as usr_site_code,
                        'Unknown' as usr_segment,
                        'Unknown' as usr_sub_segment,
                        'Unknown' as usr_country,
                        'Unknown' as usr_region,
                        'Unknown' as usr_job_family_group,
                        'Unknown' as usr_job_family,
                        'Unknown' as usr_supervisory_org,
                        'Unknown' as usr_level_2,
                        'Unknown' as usr_level_3,
                        'Unknown' as usr_level_4,
                        'Unknown' as usr_level_5,
                        'Unknown' as usr_level_6,
                        'Unknown' as usr_level_7,
                        'Unknown' as usr_level_8,
                        'no_source' as src
        FROM            df_user_list
    ),
    df_units_0 AS (
        SELECT      UNIT_SITE_CODE as unit_site_code,
                    UNIT_ID as unit_id,
                    UNIT_NAME as unit_name,
                    UNIT_LOCATION_ADDRESS_1 as unit_address,
                    UNIT_LOCATION_ADDRESS_3 as unit_city,
                    UNIT_LOCATION_ADDRESS_5 as unit_postal_code,
                    UNIT_COUNTRY as unit_country,
                    UNIT_BUSINESS_SEGMENT as unit_business_segment,
                    UNIT_CONSOLIDATION_UNIT as unit_consolidation_unit,
                    UNIT_REGION as unit_region,
                    UNIT_LATITUDE as unit_latitude,
                    UNIT_LONGITUDE as unit_longitude,
                    UNIT_SITE_TYPE as unit_site_type
        FROM        df_units
    ),
    df_web_0 AS (
        SELECT      DISTINCT
                    UNIT_COUNTRY,
                    UNIT_SITE_CODE,
                    ROW_NUMBER() OVER (PARTITION BY UNIT_COUNTRY ORDER BY UNIT_COUNTRY,UNIT_SITE_CODE) as rn
        FROM        df_units_0
        WHERE       UNIT_SITE_TYPE IN ('Office (O)','Factory and Office (F&O)')
        ORDER BY    UNIT_COUNTRY,UNIT_SITE_CODE
    ),
    df_web_1 AS (
        SELECT      UNIT_COUNTRY,
                    UNIT_SITE_CODE
        FROM        df_web_0
        WHERE       rn=1
    ),
    df_web_2 AS (
        SELECT      DISTINCT(UPPER(Country)) as Country
        FROM        df_act_web
        WHERE       metricName = 'Traffic'                
    ),
    df_web_3 AS (
        SELECT      a.Country,
                    b.*
        FROM        df_web_2 a
        LEFT JOIN   df_web_1 b
            ON      b.UNIT_COUNTRY LIKE CONCAT('%',a.Country,'%')
    ),
    df_web_4 AS (
        SELECT  *
        FROM (
            SELECT  *,
                    ROW_NUMBER() OVER (PARTITION BY Country ORDER BY Country,UNIT_SITE_CODE) as rn
            FROM    df_web_3
        )
        WHERE   rn=1
    ),
    df_web_5 AS (
        SELECT  md5(upper(Country)) as usr_id,
                upper(Country) as usr_email,
                'WebAppUsers' as usr_type,
                UNIT_SITE_CODE as usr_site_code,
                'Unknown' as usr_segment,
                'Unknown' as usr_sub_segment,
                'Unknown' as usr_country,
                'Unknown' as usr_region,
                'Unknown' as usr_job_family_group,
                'Unknown' as usr_job_family,
                'Unknown' as usr_supervisory_org,
                'Unknown' as usr_level_2,
                'Unknown' as usr_level_3,
                'Unknown' as usr_level_4,
                'Unknown' as usr_level_5,
                'Unknown' as usr_level_6,
                'Unknown' as usr_level_7,
                'Unknown' as usr_level_8,
                'no_source' as src
        FROM    df_web_4
    )

            SELECT a.* FROM df_usr a JOIN df_email_act b on a.usr_id = b.usr_id
    UNION   SELECT * FROM df_user_missing
    UNION   SELECT * FROM df_web_5
""").withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df_usr_o)

<mark>**USAGE STATISTICS - DIM UNITS**</mark>

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "output", "target_table": "usr_stats_dim_unit", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"schema":"output",      "table":"usr_stats_dim_user",                                        "view":"df_usr"},
        {"schema":"raw",         "table":"units_guide_units_details",                                 "view":"df_units"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

## ------------------------------------------------------------------------------------

df_units_o = spark.sql("""
WITH df_units_0 AS (
    SELECT      UNIT_SITE_CODE as unit_site_code,
                UNIT_ID as unit_id,
                UNIT_NAME as unit_name,
                UNIT_LOCATION_ADDRESS_1 as unit_address,
                UNIT_LOCATION_ADDRESS_3 as unit_city,
                UNIT_LOCATION_ADDRESS_5 as unit_postal_code,
                UNIT_COUNTRY as unit_country,
                UNIT_BUSINESS_SEGMENT as unit_business_segment,
                UNIT_CONSOLIDATION_UNIT as unit_consolidation_unit,
                UNIT_REGION as unit_region,
                UNIT_LATITUDE as unit_latitude,
                UNIT_LONGITUDE as unit_longitude,
                UNIT_SITE_TYPE as unit_site_type
    FROM        df_units
),
df_units_list AS (
    SELECT DISTINCT     a.usr_site_code
    FROM                df_usr a        
),
df_units_final AS (
    SELECT      a.*
    FROM        df_units_0 a
    JOIN        df_units_list b
    ON          a.unit_site_code = b.usr_site_code
)

SELECT * FROM df_units_final
""").withColumns({
        "_run_id": lit(job_id),
        "_run_timestamp" :  F.current_timestamp()
    })

## ------------------------------------------------------------------------------------
create_table(metadata_json, df_units_o)

## Send Helios Automated Data Checks report

In [ ]:
import re
import requests 
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import DateType
from datetime import date

## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema":"output", "target_table":"helios_automated_data_checks", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"schema":"output"     , "table": "fact_helios_sales_skuprint" , "view": "fact_helios_sales_skuprint"},
        {"schema":"output"     , "table": "dim_calendar_period" ,        "view": "dim_calendar_period"},
        {"schema":"output"     , "table": "dim_calendar_day" ,        "view": "dim_calendar_day"},
        {"table":"dbo.v_odsrules", "view":"v_odsrules", "linked_service_name":"DATAVERSE_SL_SQL_LS", "opertation_type":"serverless jdbc ls"}
    ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

### ------------------------------------------------------------------------------------

ods_rules_df = spark.sql("""
        SELECT      key1,
                    value1,
                    criteria                 
        FROM        v_odsrules
        WHERE       Function = "HELIOS_AUTOMATED_DATA_CHECKS"
""")

distribution_list = ods_rules_df.filter(F.col("criteria")=="DISTRIBUTION_LIST").select("value1").collect()[0][0]
print(f"Report will be sent to {distribution_list}")

send_report_on_dates = ods_rules_df.filter(F.col("criteria")=="SEND_REPORTS_ON_SPECIFIC_DATES").select("value1").collect()[0][0]

periods_to_report = ods_rules_df.filter(F.col("criteria")=="PERIODS_TO_REPORT").select("value1").collect()[0][0]
print(f"Number of periods to report {periods_to_report}")
periods_to_report = int(periods_to_report) + 1 

percent_threshold = ods_rules_df.filter(F.col("criteria")=="PERCENT_THRESHOLD").select("value1").collect()[0][0]
percent_threshold = float(str(percent_threshold).strip().replace("%", ""))
print(f"Threshold: {percent_threshold} %")

email_header = ods_rules_df.filter(F.col("criteria")=="EMAIL_HEADER").select("value1").collect()[0][0]
email_subject = ods_rules_df.filter(F.col("criteria")=="EMAIL_SUBJECT").select("value1").collect()[0][0]

today = date.today().strftime('%Y-%m-%d')
print(f"\nToday's date: {today}")

df = spark.table("dim_calendar_day")

# Convert 'day' column to Date type if it's not already
df = df.withColumn("day", F.col("day").cast(DateType()))

# Sort data by date
df_sorted = df.orderBy("day")

# Create window functions for week and day numbering
week_window = Window.partitionBy("year", "period").orderBy("week")
day_window = Window.partitionBy("year", "period", "week").orderBy("day")

# Add week and day numbers
df_with_numbers = df_sorted.withColumn(
    "week_number_in_year_period", 
    F.dense_rank().over(week_window)
).withColumn(
    "day_number_in_week", 
    F.row_number().over(day_window)
)

# Create final format: YYYYPXXWXDX
df_final = df_with_numbers.withColumn(
    "calendar_code",
    F.concat(
        F.col("year").cast("string"),
        F.col("period").cast("string"), 
        F.lit("W"),
        F.col("week_number_in_year_period").cast("string"),
        F.lit("D"),
        F.col("day_number_in_week").cast("string")
    )
)

df_result = df_final.select(
    "day", "year", "period", "week", 
    "week_number_in_year_period", 
    "day_number_in_week", 
    "calendar_code"
)

df_filtered = df_result.filter(F.col("day")==F.lit(today)).select("calendar_code")
current_period_week_day = df_filtered.collect()[0][0]
print(f"Today is {current_period_week_day}")
print(send_report_on_dates)
dates_list = [date.strip() for date in send_report_on_dates.split(',')]

# Check if today is in the list of report dates
if current_period_week_day in dates_list:
    print("Send report")

    # Create DataFrame from your SQL query
    df_dim_calendar_period = spark.sql(f"""
        SELECT      DISTINCT 
                    year_Period 
        FROM        dim_calendar_period
        WHERE       period_last_day <= '{today}'
    """)

    # Split year_Period by "P" to get year and period separately
    df_split = df_dim_calendar_period.withColumn("split_parts", F.split(F.col("year_Period"), "P"))

    df_with_parts = df_split.withColumn("year", F.col("split_parts")[0].cast("int")) \
                        .withColumn("period", F.col("split_parts")[1].cast("int"))

    # Convert to integer format: year + period with proper padding
    df_with_int_period = df_with_parts.withColumn(
        "period_int",
        (F.col("year") * 100 + F.col("period")).cast("int")
    ).select("year_Period", "year", "period", "period_int") \
    .orderBy(F.col("period_int").desc()) \
    .limit(periods_to_report)

    # Sort by integer period and use lag to find previous
    window_spec = Window.orderBy(F.col("period_int"))

    df_with_previous_int = df_with_int_period.withColumn(
        "previous_period_int", 
        F.lag(F.col("period_int"), 1).over(window_spec)
    )

    # Convert previous period back to YYYYPXX format
    df_with_previous = df_with_previous_int.withColumn(
        "previous_year_Period",
        F.when(F.col("previous_period_int").isNotNull(),
            F.concat(
                (F.col("previous_period_int") / 100).cast("int").cast("string"),
                F.lit("P"),
                F.lpad((F.col("previous_period_int") % 100).cast("string"), 2, "0")
            )
        ).otherwise(F.lit(None))
    ).select("year_Period", "year", "period", "period_int", "previous_period_int", "previous_year_Period")

    # Verify that previous period exists in original dataset
    df_dim_calendar_period_with_previous = df_with_previous.alias("current") \
        .join(df_dim_calendar_period.alias("prev"), 
            F.col("current.previous_year_Period") == F.col("prev.year_Period"), 
            "left") \
        .select(
            F.col("current.year_Period"),
            F.col("current.period_int"),
            F.col("current.previous_year_Period"),
            F.col("current.previous_period_int"),
            F.when(F.col("prev.year_Period").isNotNull(), F.lit("EXISTS"))
            .otherwise(F.lit("NOT_EXISTS")).alias("previous_exists")
        ) \
        .filter(F.col("previous_exists")=="EXISTS")

    df_dim_calendar_period_with_previous.createOrReplaceTempView("dim_calendar_period_with_previous")

    # Get your data from SQL
    df_sales = spark.sql("""
    WITH factdata AS 
    (
        SELECT      Unit_ID, Unit, concat(Year,Period) as year_Period, COALESCE(SUM(Tonnes),0) as Tonnes 
        FROM        fact_helios_sales_skuprint
        GROUP BY    Unit_ID, Unit, concat(Year,Period)
    ) 
    SELECT      f.Unit_ID,
                f.Unit,
                f.year_Period,
                f.Tonnes,
                CASE WHEN coalesce(f.Tonnes,0) + coalesce(p.Tonnes,0) = 0 THEN 0 
                        ELSE round(100.00 * (coalesce(f.Tonnes,0) - coalesce(p.Tonnes,0)) / ( ( coalesce(f.Tonnes,0) + coalesce(p.Tonnes,0)) / 2 ),2)
                END as percent
    FROM        factdata f 
    JOIN        dim_calendar_period_with_previous d 
    ON          f.year_Period = d.year_Period
    LEFT JOIN   factdata p 
    ON          f.Unit_ID = p.Unit_ID
    AND         p.year_Period  = d.previous_year_Period     
    """)

    # Get unique year_Period values for pivot
    year_periods = df_sales.select("year_Period").distinct().rdd.flatMap(lambda x: x).collect()
    year_periods = sorted(year_periods)

    print(f"Found {len(year_periods)} unique year periods")

    # Create pivot using standard PySpark method
    pivot_data = df_sales.groupBy("Unit_ID", "Unit") \
        .pivot("year_Period", year_periods) \
        .agg(
            F.first(F.col("Tonnes")).alias("Tonnes"),
            F.first(F.col("percent")).alias("percent")
        )

    # Clean data - TYLKO RAZ!
    all_columns = pivot_data.columns
    pivot_data_clean = pivot_data
    for column_name in all_columns:
        pivot_data_clean = pivot_data_clean.withColumn(
            column_name,
            F.when(F.col(column_name) == "undefined", F.lit(""))
            .when(F.col(column_name).isNull(), F.lit(""))
            .otherwise(F.col(column_name))
        )

    pivot_data_clean = pivot_data_clean.orderBy(F.col("Unit").asc())

    # Get all columns from cleaned data
    all_columns = pivot_data_clean.columns

    # Separate basic columns from data columns
    basic_columns = ["Unit_ID", "Unit"]
    data_columns = [col for col in all_columns if col not in basic_columns]

    # Extract unique periods from column names
    periods = set()
    for col in data_columns:
        if "_Tonnes" in col:
            period = col.replace("_Tonnes", "")
            periods.add(period)
        elif "_percent" in col:
            period = col.replace("_percent", "")
            periods.add(period)

    # Sort periods chronologically
    periods_sorted = sorted(list(periods))

    # Create HTML content with email-safe CSS
    html_content = """
    <style>
    table {
        border-collapse: collapse;
        width: 100%;
        font-family: Arial, sans-serif;
        font-size: 12px;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 6px;
        text-align: center;
    }
    th {
        background-color: #f2f2f2;
        font-weight: bold;
        font-size: 14px;
        color: black;
    }
    .unit-info {
        text-align: left;
        font-weight: normal;
        color: black;
    }
    .unit-id-col {
        width: 80px;
    }
    .unit-name-col {
        width: 200px;
        min-width: 200px;
    }

    tr td {
        background-color: #ffffff;
        color: black;
    }

    tr.even-row td {
        background-color: #f5f5f5;
        color: black;
    }

    .red-cell {
        background-color: #ffcccc !important;
        color: #cc0000 !important;
        font-weight: bold !important;
    }
    </style>
    <table>
    <tr>
    """

    # Add table headers
    html_content += '<th class="unit-info unit-id-col">Unit_ID</th>'
    html_content += '<th class="unit-info unit-name-col">Unit</th>'

    for i, period in enumerate(periods_sorted):
        html_content += f'<th class="tonnes-col">{period}<br>Tonnes</th>'
        html_content += f'<th class="percent-col">{period}<br>%</th>'

    html_content += "</tr>"

    # Add data rows
    row_number = 0
    styled_cells_count = 0

    for row in pivot_data_clean.collect():
        row_number += 1
        row_class = "odd-row" if row_number % 2 == 1 else "even-row"
        html_content += f'<tr class="{row_class}">'
        
        html_content += f'<td class="unit-info unit-id-col">{row["Unit_ID"] if row["Unit_ID"] is not None else ""}</td>'
        html_content += f'<td class="unit-info unit-name-col">{row["Unit"] if row["Unit"] is not None else ""}</td>'
        
        for i, period in enumerate(periods_sorted):
            # Tonnes column
            tonnes_col = f"{period}_Tonnes"
            tonnes_value = row[tonnes_col] if tonnes_col in row and row[tonnes_col] is not None else ""
            
            if tonnes_value != "":
                try:
                    formatted_tonnes = f"{float(tonnes_value):,.2f}" if float(tonnes_value) != 0 else "0.00"
                except (ValueError, TypeError):
                    formatted_tonnes = str(tonnes_value)
            else:
                formatted_tonnes = ""
            
            html_content += f'<td class="tonnes-col">{formatted_tonnes}</td>'

            # Percent column
            percent_col = f"{period}_percent"
            percent_value = row[percent_col] if percent_col in row and row[percent_col] is not None else ""
            
            # Format percentage value
            if percent_value != "":
                try:
                    formatted_percent = f"{float(percent_value):.2f}%" if float(percent_value) != 0 else "0.00%"
                    numeric_value = float(percent_value)
                    
                    # Apply red styling if value exceeds threshold
                    if numeric_value > percent_threshold or numeric_value < -percent_threshold:
                        html_content += f'<td class="percent-col red-cell">{formatted_percent}</td>'
                        styled_cells_count += 1
                    else:
                        html_content += f'<td class="percent-col">{formatted_percent}</td>'
                except (ValueError, TypeError):
                    formatted_percent = str(percent_value)
                    html_content += f'<td class="percent-col">{formatted_percent}</td>'
            else:
                html_content += f'<td class="percent-col"></td>'
        
        html_content += "</tr>"

    html_content += "</table>"

    print("HTML table generated successfully!")
    print(f"Number of rows: {pivot_data_clean.count()}")
    print(f"Number of periods: {len(periods_sorted)}")
    print(f"Periods: {periods_sorted}")
    print(f"Styled cells count: {styled_cells_count}")

    # Get workspace environment
    workspace_name = mssparkutils.env.getWorkspaceName()
    workspace_env = re.findall(r"(dev|prod|uat)(?=syn)", workspace_name)[0]
    print(f"Workspace environment: {workspace_env}")

    # Recipients
    recipients = distribution_list

    # Send notification
    send_notification(
        message=html_content, 
        subject=email_subject, 
        header=email_header, 
        status="Success", 
        color="Green", 
        recipients=recipients, 
        env=workspace_env
    )

else:
    print("No report scheduled for today")

In [22]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "process", "target_table": "calculated_tuc_temp", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"schema":"normalized",        "table":"material_unit_conversion",                                  "view":"df_uom",                "project_name":"atlas"},
        {"schema":"normalized",        "table":"uom_unit_conversion",                                       "view":"df_uom_unit_conv",      "project_name":"atlas"},
        {"schema":"normalized",        "table":"bom_consolidated",                                          "view":"df_bom_complex",        "project_name":"atlas"},
        {"schema":"output",            "table":"lorax_dim_bill_of_material_persist_history",                "view":"df_bom_hist",           "project_name":"lorax"},
        {"schema":"output",            "table":"lorax_dim_bill_of_material_persist_header",                 "view":"df_bom_hdr",            "project_name":"lorax"},
        {"schema":"output",            "table":"dim_material",                                              "view":"dim_material",          "project_name":"ods"},
        {"schema":"raw",               "table":"mdg_product_chain_bom",                                     "view":"df_pc_bom",             "project_name":"ods"}
           ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_schema_name": "'''+str(admin_schema_name)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''",
    "process_date": "'''+str(process_date)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

# Cache the product chain bom for performance reason
df_pc_bom_0 = spark.sql("""
    SELECT  a.PARENT as parent,
            a.CHILD as child,
            a.QUANTITY as qty,
            a.LEVEL as lvl
    FROM    df_pc_bom a
""")
df_pc_bom_0.createOrReplaceTempView('df_pc_bom_0')
df_pc_bom_0.cache()

## --------------------------------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------------------------------------------------------------------------------------------------
## 1. DETERMINE THE PORTFOLIO OF PRODUCTS TO CONSIDER FOR THIS REPORT. SPLIT COMPLEX vs. REGULAR MATERIALS
## --------------------------------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------------------------------------------------------------------------------------------------


MATERIAL_TYPES = ['FERT', 'ZREP']

df_mat = spark.sql(f"""SELECT * FROM dim_material WHERE material_type IN {tuple(MATERIAL_TYPES)}""")
df_mat.createOrReplaceTempView('df_mat')

print("1. DETERMINE THE PORTFOLIO")

df_product_portfolio = spark.sql("""
WITH df_bom_cplx AS (
    --Build the bom for complex items
    SELECT DISTINCT
                'COMPLEX' as src,
                b.representative_item as complex_representative_item,
                a.material_number,
                a.bom_component,
                a.component_quantity / a.base_quantity as qty
    FROM        df_bom_complex a
    LEFT JOIN   df_mat b
    ON          a.material_number = b.src_agnostic_unique_id
    WHERE       a.bom_usage = 5
    AND         a.plant IS NOT NULL
),
df_cplx_mat_list AS (
    --Get list of complex materials to be able to distinguish complex from non-complex items
    SELECT DISTINCT
                a.material_number
    FROM        df_bom_cplx a
),
df_all_materials AS (
    --Get the portfolio of products in scope of this scheme
    SELECT      DISTINCT
                b.src_agnostic_unique_id as material_number,
                b.description,
                b.representative_item,
                CASE WHEN c.material_number IS NOT NULL Then 'Y' ELSE 'N' END AS complex_flag  
    FROM        df_mat b
    LEFT JOIN   df_cplx_mat_list c
    ON          b.src_agnostic_unique_id = c.material_number
),
df_bom_hdr AS (
    --Find corresponding BOMs in BOM Version. Level = 2 
    SELECT      a.*
    FROM        df_bom_hdr a
    JOIN        df_all_materials b
    ON          a.parent = b.material_number
    WHERE       a.eff_valid_from_date <= date_format(concat(cast(date_format(current_date(),'y') as int) - 1,'-12-31'),'yyyy-MM-dd')
    AND         a.eff_valid_to_date >= date_format(concat(cast(date_format(current_date(),'y') as int) - 1,'-12-31'),'yyyy-MM-dd')
    AND         b.complex_flag = 'Y'         
),
df_bom_hst AS (
    SELECT DISTINCT
                'BOM' as src,
                '' as complex_representative_item,
                a.header_material as material_number,
                split(a.path, ',')[1] as bom_component,
                '' as qty
    FROM        df_bom_hist a
    JOIN        df_bom_hdr b
    ON          a.header_material = b.parent
    AND         a.header_plant = b.plant
    AND         a.version = b.version
    AND         a.level = 2
),
df_both AS (
SELECT * FROM df_bom_hst
UNION
SELECT * FROM df_bom_cplx
),
df_both_1 AS (
--Find corresponding representative items and EAN code
SELECT      a.*,
            b.representative_item,
            b.ean
FROM        df_both a
LEFT JOIN   df_mat b
ON          a.bom_component = b.src_agnostic_unique_id
ORDER BY    a.material_number,
            b.representative_item
),
df_pc_bom_1 AS (
    --Grab the correct zrep using product chain BOM using the component ZREP code
    SELECT      a.parent as parent_lvl2,
                a.child as child_lvl2,
                a.qty as qty_lvl2,
                b.merchandising_unit
    FROM        df_pc_bom_0 a
    LEFT JOIN   df_mat b
    ON          a.child = b.src_agnostic_unique_id
    WHERE       a.lvl = 2
),
df_pc_bom_2 AS (
    --One more iteration needed if the subsequent component ZREP is an MCU (It has another level to get to the RSU)
    SELECT      a.*,
                b.child as child_lvl3
    FROM        df_pc_bom_1 a
    LEFT JOIN   df_pc_bom_0 b
    ON          a.child_lvl2 = b.parent
    AND         b.lvl=3
    AND         a.merchandising_unit = 'X'
),
df_joined_0 AS (
    SELECT          a.*,
                    b.child_lvl2,
                    b.merchandising_unit,
                    b.child_lvl3,
                    coalesce(b.child_lvl3,b.child_lvl2,a.bom_component) as link_fert
    FROM            df_both_1 a
    LEFT JOIN       df_pc_bom_2 b
    ON              a.bom_component = b.parent_lvl2
    ORDER BY        a.material_number,coalesce(b.child_lvl2,a.bom_component)
),
df_joined AS (
    SELECT          a.*,
                    b.representative_item as link
    FROM            df_joined_0 a
    LEFT JOIN       df_mat b
    ON              a.link_fert = b.src_agnostic_unique_id
),
df_final_1 AS (
    SELECT          a.bom_component as mat_key,
                    concat(a.material_number,'-',a.bom_component) as material_number,
                    d.description,
                    d.representative_item,
                    b.ean as ean_uc_complex,
                    'Y' as complex_flag,
                    a.material_number as complex_material_number,
                    c.description as complex_material_description,
                    c.representative_item as complex_representative_item,
                    a.link
    FROM            df_joined a
    LEFT JOIN       df_mat b
    ON              a.link = b.src_agnostic_unique_id
    LEFT JOIN       df_mat c
    ON              a.material_number = c.src_agnostic_unique_id
    LEFT JOIN       df_mat d
    ON              a.bom_component = d.src_agnostic_unique_id
    WHERE             a.src = 'BOM'
),
df_final_2 AS (
    SELECT          a.*,
                    b.bom_component as complex_pricing_material_number,
                    b.qty as complex_tuc_qty
    FROM            df_final_1 a
    LEFT JOIN       df_joined b
    ON              a.link = b.link
    AND             a.complex_material_number = b.material_number
    AND             b.src = 'COMPLEX'
),
df_all AS (
    SELECT      a.mat_key,
                a.representative_item,
                a.material_number,
                a.description,
                a.ean_uc_complex,
                a.complex_flag,
                a.complex_material_number,
                a.complex_material_description,
                a.complex_representative_item,
                a.complex_pricing_material_number,
                a.complex_tuc_qty
    FROM        df_final_2 a
    UNION
    SELECT      a.material_number as mat_key,
                a.representative_item,
                a.material_number,
                a.description,
                NULL as ean_uc_complex,
                a.complex_flag,
                NULL as complex_material_number,
                NULL as complex_material_description,
                NULL as complex_representative_item,
                NULL as complex_pricing_material_number,
                NULL as complex_tuc_qty
    FROM        df_all_materials a
    WHERE       a.complex_flag = 'N'
),
df_all_final AS (
    SELECT      a.mat_key,
                a.representative_item,
                a.material_number,
                --Adding classifications needed for product code determination
                d.Business_Segment as business_segment,
                d.Tech as tech,
                d.Market_Segment as market_segment,
                d.Supply_Segment as supply_segment,
                d.Product_Category as product_category,
                d.Ingredient_Variety as ingredient_variety,
                d.Product_Type as product_type,
                lower(a.Description) as description,
                --Addition classifications needed for users to see in the Power BI report
                CASE    WHEN d.Business_Segment = 'Chocolate' and d.market_segment = 'Frozen Snacks'                THEN 'ICE CREAM ITEMS'
                        WHEN d.Business_Segment = 'Chocolate' and d.market_segment <> 'Frozen Snacks'               THEN 'CHOCO ITEMS'
                        WHEN d.Business_Segment = 'Gum and Confections' and d.market_segment <> 'Frozen Snacks'     THEN 'GUM ITEMS'   
                        WHEN d.Business_Segment = 'Petcare'                                                         THEN 'PETFOOD ITEMS'
                        WHEN d.Business_Segment = 'Food'                                                            THEN 'FOOD ITEMS'
                ELSE    d.Business_Segment
                END as item_type,
                d.Brand_Flag as brand,
                d.Consumer_Pack_Format as consumer_pack_format,
                d.Consumer_Pack_Type as consumer_pack_type,
                d.Product_Pack_Size as pack_size,
                replace(d.Multipack_Quantity,'-pack','') as multipack_quantity,
                d.Traded_Unit_Configuration as traded_unit_configuration,
                a.complex_flag,
                a.complex_material_number,
                a.complex_material_description,
                a.complex_representative_item,
                a.complex_pricing_material_number,
                a.complex_tuc_qty,
                a.ean_uc_complex
    FROM        df_all a
    LEFT JOIN   df_mat d
        ON      a.mat_key = d.src_agnostic_unique_id
)
SELECT * FROM df_all_final --where complex
""")
# display(df_product_portfolio)
df_product_portfolio.createOrReplaceTempView('df_product_portfolio')
df_product_portfolio.cache()
# record_count = df_product_portfolio.count()
# print(f"The total number of records in the DataFrame is: {record_count}")

## --------------------------------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------------------------------------------------------------------------------------------------
## 2. DETERMINE THE TRADED UNIT CONFIGURATION
## --------------------------------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------------------------------------------------------------------------------------------------
print("2. DETERMINE THE TRADED UNIT CONFIGURATION")

# Convert UOM table to proper uom unit item (TH vs TS)
df_uom = spark.sql("""
    WITH df_uom_unit_conv_2 AS (
        SELECT  Unit_of_Measurement as source_uom,
                External_Unit_of_Measurement_Commercial_Format as target_uom
        FROM    df_uom_unit_conv
        WHERE   language_key = 'EN'
    )

    SELECT  a.Material_Number as material,
            a.Alternative_Unit_Of_Measure_For_Stockkeeping_Unit,
            b.target_uom as uom,
            Denominator_For_Conversion_To_Base_Units_Of_Measure/Numerator_For_Conversion_To_Base_Units_Of_Measure as tuc_pc
    FROM    df_uom a, df_uom_unit_conv_2 b
    WHERE   a.Alternative_Unit_Of_Measure_For_Stockkeeping_Unit = b.source_uom
        AND b.target_uom = 'PC'

""")
df_uom.createOrReplaceTempView('df_uom')
df_uom.cache()

# TUC (Traded Unit Configuration) extraction patterns
# Method 1: From MARM table (tuc_pc_m1)
# Method 2: From ZZGLOBAL classification when it's numeric (tuc_pc_m2)
# Methods 3-14: Extract from description field using various patterns

patterns = [
    # UVC/UCV/UC patterns (m3-m8) - Consumer unit markers
    r"(\d+)\s+uvc",  # m3: "24 uvc" - number with space before uvc
    r"(\d+)uvc",     # m4: "24uvc" - number directly before uvc
    r"(\d+)\s+uvc",  # m5: "24 ucv" - number with space before ucv
    r"(\d+)+uvc",     # m6: "24ucv" - number directly before ucv
    r"(\d+)\s+uc",   # m7: "24 uc" - number with space before uc
    r"(\d+)uc",      # m8: "24uc" - number directly before uc
    
    # Weight-based patterns (m9-m11) - Extract from product weight notation
    #r'(\d+(?:[*x]\d+)+)[*x]\d+g',  # m9: "2*12*150g" or "2x12x150g" -> "2*12" - multi-level before weight
    #r'\d+g[*x](\d+(?:[*x]\d+)*)',  # m10: "56gx12x12" -> "12*12" - multi-level after weight
    #r'(?<![*x\d])(\d+)[*x]\d+g',   # m11: "24*400g" -> "24" - single number before weight
]

# Generate CASE statements for pattern matching with x->* normalization
pattern_cases = []
for i, pattern in enumerate(patterns, start=3):
    pattern_cases.append(f"""
                    CASE WHEN LENGTH(regexp_extract(lower(description), r"{pattern}", 1)) > 0 
                         THEN replace(regexp_extract(lower(description), r"{pattern}", 1), 'x', '*')
                         ELSE NULL
                    END AS tuc_pc_m{i}""")

pattern_sql = ",".join(pattern_cases)
coalesce_cols = ",".join(["tuc_pc_m1", "tuc_pc_m2"] + [f"tuc_pc_m{i}" for i in range(3, 3 + len(patterns))])

print(f"Tuc columns: {coalesce_cols}")
df_tuc_marm = spark.sql(f"""
    WITH df_0 AS (
        -- Method 1: Get TUC from MARM (Material Unit of Measure) table
        SELECT              a.*,
                            b.tuc_pc as tuc_pc_m1
        FROM                df_product_portfolio a
        LEFT JOIN           df_uom b
        ON                  a.mat_key = b.material
    ),
    df_1 AS (
        -- Method 2: Extract TUC from ZZGLOBAL classification when it contains only digits
        SELECT      *,
                    CASE WHEN traded_unit_configuration not rlike '[^0-9]' 
                         THEN traded_unit_configuration 
                    END AS tuc_pc_m2              
        FROM        df_0 
    ),
    df_2 AS (
        -- Methods 3-11: Extract TUC from description using regex patterns
        -- Covers: UVC/UCV/UC markers and weight-based notations (e.g., 24*400g, 2*12*150g, 56gx12x12)
        -- All separators normalized to '*' (e.g., "12x24" becomes "12*24")
        SELECT      *, {pattern_sql}
        FROM        df_1
    ),
    df_3 AS (
        -- Final selection: Add coalesced result while keeping all individual method columns for debugging
        SELECT      material_number,
                    description,
                    traded_unit_configuration,
                    coalesce({coalesce_cols}) as tuc_pc_m1
        FROM        df_2
    )
    SELECT * FROM df_3
""")
df_tuc_marm.createOrReplaceTempView('df_tuc_marm')
df_tuc_marm.cache()

# record_count = df_product_portfolio.count()
# print(f"The total number of records in df_tuc_marm is: {record_count}")

df_tuc_product_chain = spark.sql("""
    WITH df_lvl1 AS (
        --Consider only products in the portfolio
        SELECT  a.parent as material,
                a.child as lvl1_child,
                a.qty as lvl1_qty,
                a.lvl as lvl1_lvl
        FROM    df_pc_bom_0 a
        JOIN    df_product_portfolio c
        ON      c.material_number = a.parent
    ),
    df_lvl2 AS (
        SELECT      a.*,
                    b.parent as lvl2_parent,
                    b.child as lvl2_child,
                    b.qty as lvl2_qty,
                    b.lvl as lvl2_lvl
        FROM        df_lvl1 a
        LEFT JOIN   df_pc_bom_0 b
            ON      a.lvl1_child = b.parent
            AND     a.lvl1_lvl+1 = b.lvl
    ),
    df_lvl3 AS (
        SELECT      a.*,
                    b.parent as lvl3_parent,
                    b.child as lvl3_child,
                    b.qty as lvl3_qty,
                    b.lvl as lvl3_lvl
        FROM        df_lvl2 a
        LEFT JOIN   df_pc_bom_0 b
            ON      a.lvl2_child = b.parent
            AND     a.lvl2_lvl+1 = b.lvl
    ),
    df_lvl4 AS (
        SELECT      a.*,
                    b.parent as lvl4_parent,
                    b.child as lvl4_child,
                    b.qty as lvl4_qty,
                    b.lvl as lvl4_lvl
        FROM        df_lvl3 a
        LEFT JOIN   df_pc_bom_0 b
            ON      a.lvl3_child = b.parent
            AND     a.lvl3_lvl+1 = b.lvl
    ),
    df_with_rsu_flag AS (
        SELECT * FROM (
            SELECT      a.*,
                        b.retail_sales_unit as rsu_lvl1,
                        c.retail_sales_unit as rsu_lvl2,
                        d.retail_sales_unit as rsu_lvl3,
                        e.retail_sales_unit as rsu_lvl4
            FROM        df_lvl4 a
            LEFT JOIN   df_mat b
                ON      a.lvl1_child = b.src_agnostic_unique_id
            LEFT JOIN   df_mat c
                ON      a.lvl2_child = c.src_agnostic_unique_id
            LEFT JOIN   df_mat d
                ON      a.lvl3_child = d.src_agnostic_unique_id
            LEFT JOIN   df_mat e
                ON      a.lvl4_child = e.src_agnostic_unique_id
        ) WHERE coalesce(rsu_lvl1,rsu_lvl2,rsu_lvl3,rsu_lvl4) IS NOT NULL
    ),
    df_lvl1_1_tuc_0 AS (
        SELECT DISTINCT
                    a.material,
                    a.lvl1_child,
                    a.lvl1_qty,
                    a.rsu_lvl1,
                    a.rsu_lvl2
        FROM        df_with_rsu_flag a  
    ),
    df_lvl1_1_tuc AS (
        SELECT      a.material,
                    sum(lvl1_qty) as tuc_pc_m2
        FROM        df_lvl1_1_tuc_0 a
        WHERE       rsu_lvl1 = 'X'
        AND         rsu_lvl2 = 'X'
        GROUP BY    a.material
    ),
    df_lvl1_2_tuc AS (
        SELECT      a.material,
                    sum(lvl1_qty) as tuc_pc_m2
        FROM        df_lvl1_1_tuc_0 a
        WHERE       rsu_lvl1 = 'X'
        AND         rsu_lvl2 IS NULL
        AND         NOT EXISTS (SELECT 1 FROM df_lvl1_1_tuc b WHERE a.material = b.material)
        GROUP BY    a.material
    ),
    df_lvl2_tuc_0 AS (
        SELECT      a.material,
                    max(lvl1_qty) as lvl1_qty,
                    max(lvl2_qty) as lvl2_qty
        FROM        df_with_rsu_flag a
        WHERE       rsu_lvl1 IS NULL
        AND         rsu_lvl2 = 'X'
        GROUP BY    a.material
    ),
    df_lvl2_tuc AS (
        SELECT      material,
                    lvl1_qty * lvl2_qty as tuc_pc_m2
        FROM        df_lvl2_tuc_0
    ),
    df_final AS (
    SELECT * FROM df_lvl1_1_tuc
    UNION
    SELECT * FROM df_lvl1_2_tuc
    UNION
    SELECT * FROM df_lvl2_tuc
    )
    SELECT * FROM df_final
    """)
# display(df_tuc_product_chain)
df_tuc_product_chain.createOrReplaceTempView('df_tuc_product_chain')
df_tuc_product_chain.cache()
# record_count = df_tuc_product_chain.count()
# print(f"The total number of records in df_tuc_product_chain is: {record_count}")

df_product_portfolio_1 = spark.sql("""
WITH df_0 AS (
        SELECT      a.material_number,
                    coalesce(a.tuc_pc_m1,b.tuc_pc_m2) as tuc_pc
        FROM        df_tuc_marm a
        LEFT JOIN   df_tuc_product_chain b
        ON          a.material_number = b.material
    ),
    df_1 AS (
        SELECT      a.*,
                    coalesce (a.complex_tuc_qty,b.tuc_pc) as tuc_pc
        FROM        df_product_portfolio a
        LEFT JOIN   df_0 b
        ON          a.material_number = b.material_number
    )
    SELECT * FROM df_1
""")
# display(df_product_portfolio_1)
df_product_portfolio_1.createOrReplaceTempView('df_product_portfolio_1')
df_product_portfolio_1.cache()
# record_count = df_product_portfolio_1.count()
# print(f"The total number of records in the DataFrame is: {record_count}")

## --------------------------------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------------------------------------------------------------------------------------------------
## 3. EAN UC DETERMINATION (Using Product Chain BOM to determine the EAN of the RSU). ONLY FOR REGULAR MATERIALS
## --------------------------------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------------------------------------------------------------------------------------------------
print("3. DETERMINE THE EAN UC")

df_product_portfolio_2 = spark.sql("""
    WITH df_lvl1 AS (
        --Consider only products in the portfolio
        SELECT  a.parent as material,
                a.child as lvl1_child,
                a.qty as lvl1_qty,
                a.lvl as lvl1_lvl
        FROM    df_pc_bom_0 a
        JOIN    df_product_portfolio_1 c
            ON  c.mat_key = a.parent
    ),
    df_lvl2 AS (
        SELECT      a.*,
                    b.parent as lvl2_parent,
                    b.child as lvl2_child,
                    b.qty as lvl2_qty,
                    b.lvl as lvl2_lvl
        FROM        df_lvl1 a
        LEFT JOIN   df_pc_bom_0 b
            ON      a.lvl1_child = b.parent
            AND     a.lvl1_lvl+1 = b.lvl
    ),
    df_lvl3 AS (
        SELECT      a.*,
                    b.parent as lvl3_parent,
                    b.child as lvl3_child,
                    b.qty as lvl3_qty,
                    b.lvl as lvl3_lvl
        FROM        df_lvl2 a
        LEFT JOIN   df_pc_bom_0 b
            ON      a.lvl2_child = b.parent
            AND     a.lvl2_lvl+1 = b.lvl
    ),
    df_lvl4 AS (
        SELECT      a.*,
                    b.parent as lvl4_parent,
                    b.child as lvl4_child,
                    b.qty as lvl4_qty,
                    b.lvl as lvl4_lvl
        FROM        df_lvl3 a
        LEFT JOIN   df_pc_bom_0 b
            ON      a.lvl3_child = b.parent
            AND     a.lvl3_lvl+1 = b.lvl
    ),
    df_with_ean AS (
        SELECT      a.*,
                    CASE WHEN b.retail_sales_unit = 'X' THEN b.ean ELSE NULL END AS ean_lvl1_child,
                    CASE WHEN c.retail_sales_unit = 'X' THEN c.ean ELSE NULL END AS ean_lvl2_child,
                    CASE WHEN d.retail_sales_unit = 'X' THEN d.ean ELSE NULL END AS ean_lvl3_child,
                    CASE WHEN e.retail_sales_unit = 'X' THEN e.ean ELSE NULL END AS ean_lvl4_child
        FROM        df_lvl4 a
        LEFT JOIN   df_mat b
            ON      a.lvl1_child = b.src_agnostic_unique_id
        LEFT JOIN   df_mat c
            ON      a.lvl2_child = c.src_agnostic_unique_id
        LEFT JOIN   df_mat d
            ON      a.lvl3_child = d.src_agnostic_unique_id
        LEFT JOIN   df_mat e
            ON      a.lvl4_child = e.src_agnostic_unique_id
    ),
    df_0 AS (
        SELECT DISTINCT      
                    material,
                    coalesce(ean_lvl1_child,ean_lvl2_child,ean_lvl3_child,ean_lvl4_child) as ean_uc_regular
        FROM        df_with_ean
        WHERE       coalesce(ean_lvl1_child,ean_lvl2_child,ean_lvl3_child,ean_lvl4_child) IS NOT NULL
    ),
    df_1 AS (
        SELECT      material,
                    count(ean_uc_regular) as cnt
        FROM        df_0
        GROUP BY    material
    ),
    df_2 AS (
        SELECT      *
        FROM        (
                    SELECT      a.*,
                                b.cnt
                    FROM        df_0 a
                    LEFT JOIN   df_1 b
                    ON          a.material = b.material
                    )
        WHERE       cnt = 1
    )
    SELECT      a.*,
                coalesce(a.ean_uc_complex,b.ean_uc_regular) as ean_uc
    FROM        df_product_portfolio_1 a
    LEFT JOIN   df_2 b
    ON          a.material_number = b.material
""")
df_product_portfolio_2 = df_product_portfolio_2.drop('ean_uc_regular','ean_uc_complex')
# display(df_product_portfolio_2)
df_product_portfolio_2.createOrReplaceTempView('df_product_portfolio_2')
df_product_portfolio_2.cache()
# record_count = df_product_portfolio_3.count()
# print(f"The total number of records in the DataFrame is: {record_count}")

print("4. CREATE TABLE")
## ------------------------------------------------------------------------------------
# full_output_path = 'abfss://files@globalxsegcor1051devsa.dfs.core.windows.net/test/ean_uc'
# df_tuc_product_chain.coalesce(1).write.option("header", "true").mode("overwrite").csv(full_output_path, sep='|')
create_table(metadata_json, df_product_portfolio_2)


In [24]:
%%sql 


SELECT * FROM 
ods_process.calculated_tuc_temp
where mat_key in 
('BA02N',
'CM96H',
'CR93M',
'CS63A',
'CT78D',
'DC95J',
'DH16H',
'DH21T',
'DH48A',
'DH95F',
'DJ31C',
'DN17X',
'DP75Y',
'DR67V',
'FA12S',
'FA27J',
'FA36J',
'FB97X',
'FC57G',
'FE49H',
'FE50E',
'FF00S',
'VC475',
'VC939',
'WE680',
'YV349',
'YV709')

In [23]:
%%sql 

SELECT DISTINCT m.material TUC FROM lorax_output.lorax_dim_material m 
left join ods_process.calculated_tuc_temp t 
ON t.mat_key = m.material
WHERE m.material_type in ('FERT','ZREP') 
and m.SourceSystemUnit = 1
and t.tuc_pc is null

In [25]:

patterns = [
    # UVC/UCV/UC patterns (m3-m8) - Consumer unit markers
    r"(\d+)\s+uvc",  # m3: "24 uvc" - number with space before uvc
    r"(\d+)uvc",     # m4: "24uvc" - number directly before uvc
    r"(\d+)\s+ucv",  # m5: "24 ucv" - number with space before ucv
    r"(\d+)ucv",     # m6: "24ucv" - number directly before ucv
    r"(\d+)\s+uc",   # m7: "24 uc" - number with space before uc
    r"(\d+)uc",      # m8: "24uc" - number directly before uc
    
    # Weight-based patterns (m9-m11) - Extract from product weight notation
    #r'(\d+(?:[*x]\d+)+)[*x]\d+g',  # m9: "2*12*150g" or "2x12x150g" -> "2*12" - multi-level before weight
    #r'\d+g[*x](\d+(?:[*x]\d+)*)',  # m10: "56gx12x12" -> "12*12" - multi-level after weight
    #r'(?<![*x\d])(\d+)[*x]\d+g',   # m11: "24*400g" -> "24" - single number before weight
]

# Generate CASE statements for pattern matching with x->* normalization
pattern_cases = []
for i, pattern in enumerate(patterns, start=3):
    pattern_cases.append(f"""
                    CASE WHEN LENGTH(regexp_extract(lower(description), r"{pattern}", 1)) > 0 
                         THEN replace(regexp_extract(lower(description), r"{pattern}", 1), 'x', '*')
                         ELSE NULL
                    END AS tuc_pc_m{i}""")

pattern_sql = ",".join(pattern_cases)
coalesce_cols = ",".join(["tuc_pc_m1", "tuc_pc_m2"] + [f"tuc_pc_m{i}" for i in range(3, 3 + len(patterns))])

print(f"Tuc columns: {coalesce_cols}")
df_tuc_marm = spark.sql(f"""
    WITH df_0 AS (
        -- Method 1: Get TUC from MARM (Material Unit of Measure) table
        SELECT              a.*,
                            b.tuc_pc as tuc_pc_m1
        FROM                df_product_portfolio a
        LEFT JOIN           df_uom b
        ON                  a.mat_key = b.material
    ),
    df_1 AS (
        -- Method 2: Extract TUC from ZZGLOBAL classification when it contains only digits
        SELECT      *,
                    CASE WHEN traded_unit_configuration not rlike '[^0-9]' 
                         THEN traded_unit_configuration 
                    END AS tuc_pc_m2              
        FROM        df_0 
    ),
    df_2 AS (
        -- Methods 3-11: Extract TUC from description using regex patterns
        -- Covers: UVC/UCV/UC markers and weight-based notations (e.g., 24*400g, 2*12*150g, 56gx12x12)
        -- All separators normalized to '*' (e.g., "12x24" becomes "12*24")
        SELECT      *, {pattern_sql}
        FROM        df_1
    ),
    df_3 AS (
        -- Final selection: Add coalesced result while keeping all individual method columns for debugging
        SELECT      material_number,
                    description,
                    traded_unit_configuration,
                    {coalesce_cols} as tuc_pc_m1
        FROM        df_2
    )
    SELECT * FROM df_3
""")
df_tuc_marm.createOrReplaceTempView('df_tuc_marm')
df_tuc_marm.cache()

In [29]:
%%sql 


SELECT traded_unit_configuration, count(*) FROM 
df_tuc_marm
where material_number in 
('BA02N',
'CM96H',
'CR93M',
'CS63A',
'CT78D',
'DC95J',
'DH16H',
'DH21T',
'DH48A',
'DH95F',
'DJ31C',
'DN17X',
'DP75Y',
'DR67V',
'FA12S',
'FA27J',
'FA36J',
'FB97X',
'FC57G',
'FE49H',
'FE50E',
'FF00S',
'VC475',
'VC939',
'WE680',
'YV349',
'YV709',
'BN96Y',
'CF13X',
'CJ50J',
'CR93K',
'CR94N',
'CS53A',
'CX39B',
'DC32E',
'DD64F',
'DF86B',
'DF92T',
'DG32J',
'DH39P',
'DH84X',
'DH96S',
'DK41H',
'DN78K',
'DP52H',
'FB45X',
'FC87C',
'FE41M',
'FF50W',
'CR25D',
'CR60P',
'CX39E',
'DE87K',
'DF92W',
'DF93L',
'DH22P',
'DH53J',
'DH58G',
'DH84N',
'DJ34H',
'DK71J',
'DK82G',
'DK82H',
'DP31K',
'DP33N',
'DP75W',
'FA46C',
'FB03W',
'FB21B',
'FC55B',
'FE94W',
'VC320',
'BS23T',
'BS97M',
'CH52F',
'CH52K',
'CL12C',
'DJ96W',
'DM95A',
'DM97D',
'DP32T',
'DP39E',
'DP56A',
'FC26E',
'FD94H',
'FF00B',
'FF00H',
'CG02B',
'CH13E',
'CR93G',
'CR94C',
'CR94P',
'CS52S',
'CS83Y',
'CS85G',
'CS85V',
'DC31H',
'DC31W',
'DD21E',
'DD63N',
'DE87L',
'DH58J',
'DH95D',
'DH95K',
'DH95S',
'DJ93M',
'DP33R',
'DP48V',
'DP86W',
'FA35W',
'FB04W',
'FC05X',
'FC28S',
'FC59J',
'FE99X',
'FF00C',
'FF70S',
'AV14G',
'BR60A',
'CV36P',
'CX16C',
'DA02Y',
'DH28J',
'DJ90B',
'DK71Y',
'DM96C',
'FA09N',
'FB07S',
'FC55C',
'FE45G',
'FE99Y',
'FF00K',
'BA02M',
'BP47X',
'BR24E',
'CS69K',
'CS84A',
'CT26V',
'CX16D',
'DE79E',
'DH29B',
'DK71V',
'DP29M',
'DP33P',
'DP46S',
'DP52J',
'FA41M',
'FB04M',
'FB04Y',
'FB42Y',
'FB94G',
'FC16J',
'FC26P',
'FC98D',
'FE45E',
'FE50B',
'FE50F',
'FE94V',
'FF00A',
'FF89S',
'AF35P',
'BN98E',
'BY24F',
'CH00N',
'CH52J',
'CR94E',
'CW06J',
'DC31J',
'DE77A',
'DE79H',
'DE87H',
'DF92X',
'DF93G',
'DH15R',
'DH39F',
'DH58E',
'DK41R',
'DL46R',
'DP32X',
'DP33D',
'DP38N',
'DR67H',
'DR67P',
'FA39H',
'FB04S',
'FB10S',
'FC54R',
'FD36E',
'FE46W',
'FE99T',
'FE99V',
'FF00D',
'FF00F',
'FF11E',
'WD262')
group by traded_unit_configuration

In [ ]:
%%sql 

SELECT 
            DISTINCT m.material, m.material_type, m.Description, d.Traded_Unit_Configuration, 
            d.X_Plant_Status_Desc, d.Merchandising_Unit, d.Retail_Sales_Unit, d.Traded_Unit, 
            CASE WHEN bp.parent IS NOT NULL THEN 'X' END as product_chain_parent, 
            CASE WHEN  bc.child  IS NOT NULL THEN 'X' END as product_chain_child
FROM        lorax_output.lorax_dim_material m 
left join   df_product_portfolio_2 t 
    ON      t.mat_key = m.material
left join   ods_output.dim_material d 
    ON      m.material = d.material
LEFT JOIN   df_pc_bom bp 
    ON      bp.parent = m.material
LEFT JOIN   df_pc_bom bc 
    ON      bc.child = m.material
WHERE       m.material_type in ('FERT','ZREP') 
    and     m.SourceSystemUnit = 1
    and     t.tuc_pc is null 

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "process", "target_table": "lorax_'''+str(EPR_SCHEME).lower()+'''_step1", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"schema":"normalized",        "table":"bom_consolidated",                                          "view":"df_bom_complex",        "project_name":"atlas"},
        {"schema":"output",            "table":"dim_material",                                              "view":"df_mat",                "project_name":"ods"},
        {"schema":"raw",               "table":"mdg_product_chain_bom",                                     "view":"df_pc_bom",             "project_name":"ods"},
        {"schema":"output",            "table":"lorax_dim_bill_of_material_persist_history",                "view":"df_bom_hist"},
        {"schema":"output",            "table":"lorax_dim_bill_of_material_persist_header",                 "view":"df_bom_hdr"},
        {"schema":"process",           "table":"lorax_fact_shipment_actuals",                               "view":"df_sales"},
        {"view_name":"df_portfolio", "path":"/EPR/EPR_PORTFOLIO_FRANCE.csv", "file_format":"csv", "container":"files" ,"linked_service_name":"SOLUTION_ADLS_LS", "opertation_type":"Linked Service", "csv_options":{ "inferSchema": "true", "header": "true"}}         
           ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_schema_name": "'''+str(admin_schema_name)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''",
    "process_date": "'''+str(process_date)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

# Cache the product chain bom for performance reason
df_pc_bom_0 = spark.sql("""
    SELECT  REPLACE(LTRIM(REPLACE(a.PARENT,'0',' ')),' ','0') as parent,
            REPLACE(LTRIM(REPLACE(a.CHILD,'0',' ')),' ','0') as child,
            a.QUANTITY as qty,
            a.LEVEL as lvl
    FROM    df_pc_bom a
""")
df_pc_bom_0.createOrReplaceTempView('df_pc_bom_0')
df_pc_bom_0.cache()

## --------------------------------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------------------------------------------------------------------------------------------------
## STEP 1. DETERMINE THE PORTFOLIO OF PRODUCTS TO CONSIDER FOR THIS REPORT. SPLIT COMPLEX vs. REGULAR MATERIALS
## --------------------------------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------------------------------------------------------------------------------------------------
print("1. DETERMINE THE PORTFOLIO")

df_product_portfolio = spark.sql("""
WITH df_bom_cplx AS (
    --Build the bom for complex items
    SELECT DISTINCT
                'COMPLEX' as src,
                b.representative_item as complex_representative_item,
                a.material_number,
                a.bom_component,
                a.component_quantity / a.base_quantity as qty
    FROM        df_bom_complex a
    LEFT JOIN   df_mat b
    ON          a.material_number = b.src_agnostic_unique_id
    WHERE       a.bom_usage = 5
    AND         a.plant IS NOT NULL
),
df_cplx_mat_list AS (
    --Get list of complex materials to be able to distinguish complex from non-complex items
    SELECT DISTINCT
                a.material_number
    FROM        df_bom_cplx a
),
df_all_materials AS (
    --Get the portfolio of products in scope of this scheme
    SELECT      DISTINCT
                a.material_number,
                b.description,
                b.representative_item,
                CASE WHEN c.material_number IS NOT NULL Then 'Y' ELSE 'N' END AS complex_flag  
    FROM        df_sales a
    LEFT JOIN   df_mat b
    ON          a.material_number = b.src_agnostic_unique_id
    LEFT JOIN   df_cplx_mat_list c
    ON          a.material_number = c.material_number
    WHERE       a.sales_organization in ('"""+SALES_ORG_STR+"""')
    AND         a.flow in ('"""+FLOW_STR+"""')
    AND         year(a.actual_gi_date) >= year(current_date()) - 1
    AND         b.traded_unit = 'X'
),
df_bom_hdr AS (
    --Find corresponding BOMs in BOM Version. Level = 2 
    SELECT      a.*
    FROM        df_bom_hdr a
    JOIN        df_all_materials b
    ON          a.parent = b.material_number
    WHERE       a.eff_valid_from_date <= date_format(concat(cast(date_format(current_date(),'y') as int) - 1,'-12-31'),'yyyy-MM-dd')
    AND         a.eff_valid_to_date >= date_format(concat(cast(date_format(current_date(),'y') as int) - 1,'-12-31'),'yyyy-MM-dd')
    AND         b.complex_flag = 'Y'         
),
df_bom_hst AS (
    SELECT DISTINCT
                'BOM' as src,
                '' as complex_representative_item,
                a.header_material as material_number,
                split(a.path, ',')[1] as bom_component,
                '' as qty
    FROM        df_bom_hist a
    JOIN        df_bom_hdr b
    ON          a.header_material = b.parent
    AND         a.header_plant = b.plant
    AND         a.version = b.version
    AND         a.level = 2
),
df_both AS (
SELECT * FROM df_bom_hst
UNION
SELECT * FROM df_bom_cplx
),
df_both_1 AS (
--Find corresponding representative items and EAN code
SELECT      a.*,
            b.representative_item,
            b.ean
FROM        df_both a
LEFT JOIN   df_mat b
ON          a.bom_component = b.src_agnostic_unique_id
ORDER BY    a.material_number,
            b.representative_item
),
df_pc_bom_1 AS (
    --Grab the correct zrep using product chain BOM using the component ZREP code
    SELECT      a.parent as parent_lvl2,
                a.child as child_lvl2,
                a.qty as qty_lvl2,
                b.merchandising_unit
    FROM        df_pc_bom_0 a
    LEFT JOIN   df_mat b
    ON          a.child = b.src_agnostic_unique_id
    WHERE       a.lvl = 2
),
df_pc_bom_2 AS (
    --One more iteration needed if the subsequent component ZREP is an MCU (It has another level to get to the RSU)
    SELECT      a.*,
                b.child as child_lvl3
    FROM        df_pc_bom_1 a
    LEFT JOIN   df_pc_bom_0 b
    ON          a.child_lvl2 = b.parent
    AND         b.lvl=3
    AND         a.merchandising_unit = 'X'
),
df_joined_0 AS (
    SELECT          a.*,
                    b.child_lvl2,
                    b.merchandising_unit,
                    b.child_lvl3,
                    coalesce(b.child_lvl3,b.child_lvl2,a.bom_component) as link_fert
    FROM            df_both_1 a
    LEFT JOIN       df_pc_bom_2 b
    ON              a.bom_component = b.parent_lvl2
    ORDER BY        a.material_number,coalesce(b.child_lvl2,a.bom_component)
),
df_joined AS (
    SELECT          a.*,
                    b.representative_item as link
    FROM            df_joined_0 a
    LEFT JOIN       df_mat b
    ON              a.link_fert = b.src_agnostic_unique_id
),
df_final_1 AS (
    SELECT          a.bom_component as mat_key,
                    concat(a.material_number,'-',a.bom_component) as material_number,
                    d.description,
                    d.representative_item,
                    b.ean as ean_uc_complex,
                    'Y' as complex_flag,
                    a.material_number as complex_material_number,
                    c.description as complex_material_description,
                    c.representative_item as complex_representative_item,
                    a.link
    FROM            df_joined a
    LEFT JOIN       df_mat b
    ON              a.link = b.src_agnostic_unique_id
    LEFT JOIN       df_mat c
    ON              a.material_number = c.src_agnostic_unique_id
    LEFT JOIN       df_mat d
    ON              a.bom_component = d.src_agnostic_unique_id
    WHERE             a.src = 'BOM'
),
df_final_2 AS (
    SELECT          a.*,
                    b.bom_component as complex_pricing_material_number,
                    b.qty as complex_tuc_qty
    FROM            df_final_1 a
    LEFT JOIN       df_joined b
    ON              a.link = b.link
    AND             a.complex_material_number = b.material_number
    AND             b.src = 'COMPLEX'
),
df_all AS (
    SELECT      a.mat_key,
                a.representative_item,
                a.material_number,
                a.description,
                a.ean_uc_complex,
                a.complex_flag,
                a.complex_material_number,
                a.complex_material_description,
                a.complex_representative_item,
                a.complex_pricing_material_number,
                a.complex_tuc_qty
    FROM        df_final_2 a
    UNION
    SELECT      a.material_number as mat_key,
                a.representative_item,
                a.material_number,
                a.description,
                NULL as ean_uc_complex,
                a.complex_flag,
                NULL as complex_material_number,
                NULL as complex_material_description,
                NULL as complex_representative_item,
                NULL as complex_pricing_material_number,
                NULL as complex_tuc_qty
    FROM        df_all_materials a
    WHERE       a.complex_flag = 'N'
),
df_all_final AS (
    SELECT      a.mat_key,
                a.representative_item,
                a.material_number,
                --Adding classifications needed for product code determination
                d.Business_Segment as business_segment,
                d.Tech as tech,
                d.Market_Segment as market_segment,
                d.Supply_Segment as supply_segment,
                d.Product_Category as product_category,
                d.Ingredient_Variety as ingredient_variety,
                d.Product_Type as product_type,
                lower(a.Description) as description,
                --Addition classifications needed for users to see in the Power BI report
                CASE    WHEN d.Business_Segment = 'Chocolate' and d.market_segment = 'Frozen Snacks'                THEN 'ICE CREAM ITEMS'
                        WHEN d.Business_Segment = 'Chocolate' and d.market_segment <> 'Frozen Snacks'               THEN 'CHOCO ITEMS'
                        WHEN d.Business_Segment = 'Gum and Confections' and d.market_segment <> 'Frozen Snacks'     THEN 'GUM ITEMS'   
                        WHEN d.Business_Segment = 'Petcare'                                                         THEN 'PETFOOD ITEMS'
                        WHEN d.Business_Segment = 'Food'                                                            THEN 'FOOD ITEMS'
                ELSE    d.Business_Segment
                END as item_type,
                d.Brand_Flag as brand,
                d.Consumer_Pack_Format as consumer_pack_format,
                d.Consumer_Pack_Type as consumer_pack_type,
                d.Product_Pack_Size as pack_size,
                replace(d.Multipack_Quantity,'-pack','') as multipack_quantity,
                d.Traded_Unit_Configuration as traded_unit_configuration,
                a.complex_flag,
                a.complex_material_number,
                a.complex_material_description,
                a.complex_representative_item,
                a.complex_pricing_material_number,
                a.complex_tuc_qty,
                a.ean_uc_complex,
                c.uc as uc_qty_as_is
    FROM        df_all a
    LEFT JOIN   df_mat d
        ON      a.mat_key = d.src_agnostic_unique_id
    LEFT JOIN   df_portfolio c
        ON      a.mat_key = c.material
)
SELECT * FROM df_all_final
""")
# display(df_product_portfolio)

print("2. CREATE TABLE")
create_table(metadata_json, df_product_portfolio)

In [ ]:
## ------------------------------------------------------------------------------------
metadata_json = '''
{
    "target_table": {"target_schema": "process", "target_table": "lorax_'''+str(EPR_SCHEME).lower()+'''_step2", "format":"delta","opertation_type":"Managed Table Type"},
    "source_tables": [
        {"schema":"process",           "table":"lorax_'''+str(EPR_SCHEME).lower()+'''_step1",               "view":"df_product_portfolio"},
        {"schema":"normalized",        "table":"material_unit_conversion",                                  "view":"df_uom",                "project_name":"atlas"},
        {"schema":"normalized",        "table":"uom_unit_conversion",                                       "view":"df_uom_unit_conv",      "project_name":"atlas"},
        {"schema":"output",            "table":"dim_material",                                              "view":"df_mat",                "project_name":"ods"},
        {"schema":"raw",               "table":"mdg_product_chain_bom",                                     "view":"df_pc_bom",             "project_name":"ods"}
           ],
    "env": "'''+str(ENV)+'''",
    "project_name": "'''+str(PROJECT_NAME)+'''",
    "admin_schema_name": "'''+str(admin_schema_name)+'''",
    "admin_table_name_to_output": "'''+str(admin_table_name_to_output)+'''",
    "process_date": "'''+str(process_date)+'''"
}
'''
#Instantiate temp vies and define target table (with env)
make_env_tables(metadata_json)

# Cache the product chain bom for performance reason
df_pc_bom_0 = spark.sql("""
    SELECT  REPLACE(LTRIM(REPLACE(a.PARENT,'0',' ')),' ','0') as parent,
            REPLACE(LTRIM(REPLACE(a.CHILD,'0',' ')),' ','0') as child,
            a.QUANTITY as qty,
            a.LEVEL as lvl
    FROM    df_pc_bom a
""")
df_pc_bom_0.createOrReplaceTempView('df_pc_bom_0')
df_pc_bom_0.cache()

## --------------------------------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------------------------------------------------------------------------------------------------
## STEP 2. DETERMINE THE TRADED UNIT CONFIGURATION
## --------------------------------------------------------------------------------------------------------------------------------------------------
## --------------------------------------------------------------------------------------------------------------------------------------------------
print("1. DETERMINE THE TRADED UNIT CONFIGURATION")

# Convert UOM table to proper uom unit item (TH vs TS). Deal with IP UOM.
df_uom = spark.sql("""
    WITH df_uom_unit_conv_2 AS (
        SELECT  Unit_of_Measurement as source_uom,
                External_Unit_of_Measurement_Commercial_Format as target_uom
        FROM    df_uom_unit_conv
        WHERE   language_key = 'EN'
    ),
	df_uom_1 AS (
	    SELECT      a.Material_Number as material,
                    a.Alternative_Unit_Of_Measure_For_Stockkeeping_Unit,
                    b.target_uom as uom,
                    Denominator_For_Conversion_To_Base_Units_Of_Measure/Numerator_For_Conversion_To_Base_Units_Of_Measure as tuc_pc
		FROM        df_uom a, df_uom_unit_conv_2 b
		WHERE       a.Alternative_Unit_Of_Measure_For_Stockkeeping_Unit = b.source_uom
        AND         b.target_uom IN ('PC','IP')
	),
    df_uom_pc AS (
        SELECT      *
        FROM        df_uom_1
        WHERE       uom = 'PC'
    ),
    df_uom_ip AS (
        SELECT      *
        FROM        df_uom_1
        WHERE       uom = 'IP'
    ),
    df_uom_2 AS (
        SELECT 			a.material,
                        a.traded_unit_configuration as tuc,
                        b.tuc_pc as tuc_pc,
                        c.tuc_pc as tuc_ip
        FROM            df_mat a
        LEFT JOIN		df_uom_pc b
        ON				a.material = b.material
        LEFT JOIN		df_uom_ip c
        ON				a.material = c.material
        WHERE			a.material_type = 'FERT'
        AND             a.sourcesystemunit = 1
        --AND             a.material = 'DJ54E'
    )
    SELECT              a.material,
                        CASE WHEN a.tuc = a.tuc_ip THEN a.tuc_ip ELSE a.tuc_pc END AS tuc_pc
    FROM                df_uom_2 a
""")
# display(df_uom)
df_uom.createOrReplaceTempView('df_uom')
df_uom.cache()

df_tuc_marm = spark.sql("""
    WITH df_0 AS (
        --Method 1: Trade tuc from MARM
        SELECT              a.*,
                            b.tuc_pc as tuc_pc_m1
        FROM                df_product_portfolio a
        LEFT JOIN           df_uom b
        ON                  a.mat_key = b.material
    ),
    df_1 AS (
        --Method 2: Take tuc from ZZGLOBAL tuc classification when classification is a number
        SELECT      *,
                    CASE WHEN traded_unit_configuration not rlike '[^0-9]' THEN traded_unit_configuration END AS tuc_pc_m2              
        FROM        df_0 
    ),
    df_2 AS (
        --Method 3: Grab from description
        SELECT      *,
                    CASE WHEN LENGTH(regexp_extract(lower(description), r"(\d+)\s+uvc", 1)) > 0 
                         THEN regexp_extract(lower(description), r"(\d+)\s+uvc", 1)
                         ELSE NULL
                    END AS tuc_pc_m3,
                    CASE WHEN LENGTH(regexp_extract(lower(description), r"(\d+)uvc", 1)) > 0
                        THEN regexp_extract(lower(description), r"(\d+)uvc", 1)
                        ELSE NULL
                    END AS tuc_pc_m4,
                    CASE WHEN LENGTH(regexp_extract(lower(description), r"(\d+)\s+ucv", 1)) > 0
                        THEN regexp_extract(lower(description), r"(\d+)\s+ucv", 1)
                        ELSE NULL
                    END AS tuc_pc_m5,
                    CASE WHEN LENGTH(regexp_extract(lower(description), r"(\d+)+ucv", 1)) > 0
                        THEN regexp_extract(lower(description), r"(\d+)+ucv", 1)
                        ELSE NULL
                    END AS tuc_pc_m6,
                    CASE WHEN LENGTH(regexp_extract(lower(description), r"(\d+)\s+uc", 1)) > 0
                        THEN regexp_extract(lower(description), r"(\d+)\s+uc", 1)
                        ELSE NULL
                    END AS tuc_pc_m7,
                    CASE WHEN LENGTH(regexp_extract(lower(description), r"(\d+)+uc", 1)) > 0
                        THEN regexp_extract(lower(description), r"(\d+)+uc", 1)
                        ELSE NULL
                    END AS tuc_pc_m8
        FROM        df_1
    ),
    df_3 AS (
        SELECT      a.material_number,
                    a.description,
                    a.traded_unit_configuration,
                    coalesce(tuc_pc_m1,tuc_pc_m2,tuc_pc_m3,tuc_pc_m4,tuc_pc_m5,tuc_pc_m6,tuc_pc_m7,tuc_pc_m8) as tuc_pc_m1
        FROM        df_2 a
    )
        SELECT * FROM df_3
""")
# display(df_tuc_marm)
df_tuc_marm.createOrReplaceTempView('df_tuc_marm')
df_tuc_marm.cache()
# record_count = df_product_portfolio.count()
# print(f"The total number of records in df_tuc_marm is: {record_count}")

df_tuc_product_chain = spark.sql("""
    WITH df_lvl1 AS (
        --Consider only products in the portfolio
        SELECT  a.parent as material,
                a.child as lvl1_child,
                a.qty as lvl1_qty,
                a.lvl as lvl1_lvl
        FROM    df_pc_bom_0 a
        JOIN    df_product_portfolio c
        ON      c.material_number = a.parent
    ),
    df_lvl2 AS (
        SELECT      a.*,
                    b.parent as lvl2_parent,
                    b.child as lvl2_child,
                    b.qty as lvl2_qty,
                    b.lvl as lvl2_lvl
        FROM        df_lvl1 a
        LEFT JOIN   df_pc_bom_0 b
            ON      a.lvl1_child = b.parent
            AND     a.lvl1_lvl+1 = b.lvl
    ),
    df_lvl3 AS (
        SELECT      a.*,
                    b.parent as lvl3_parent,
                    b.child as lvl3_child,
                    b.qty as lvl3_qty,
                    b.lvl as lvl3_lvl
        FROM        df_lvl2 a
        LEFT JOIN   df_pc_bom_0 b
            ON      a.lvl2_child = b.parent
            AND     a.lvl2_lvl+1 = b.lvl
    ),
    df_lvl4 AS (
        SELECT      a.*,
                    b.parent as lvl4_parent,
                    b.child as lvl4_child,
                    b.qty as lvl4_qty,
                    b.lvl as lvl4_lvl
        FROM        df_lvl3 a
        LEFT JOIN   df_pc_bom_0 b
            ON      a.lvl3_child = b.parent
            AND     a.lvl3_lvl+1 = b.lvl
    ),
    df_with_rsu_flag AS (
        SELECT * FROM (
            SELECT      a.*,
                        b.retail_sales_unit as rsu_lvl1,
                        c.retail_sales_unit as rsu_lvl2,
                        d.retail_sales_unit as rsu_lvl3,
                        e.retail_sales_unit as rsu_lvl4
            FROM        df_lvl4 a
            LEFT JOIN   df_mat b
                ON      a.lvl1_child = b.src_agnostic_unique_id
            LEFT JOIN   df_mat c
                ON      a.lvl2_child = c.src_agnostic_unique_id
            LEFT JOIN   df_mat d
                ON      a.lvl3_child = d.src_agnostic_unique_id
            LEFT JOIN   df_mat e
                ON      a.lvl4_child = e.src_agnostic_unique_id
        ) WHERE coalesce(rsu_lvl1,rsu_lvl2,rsu_lvl3,rsu_lvl4) IS NOT NULL
    ),
    df_lvl1_1_tuc_0 AS (
        SELECT DISTINCT
                    a.material,
                    a.lvl1_child,
                    a.lvl1_qty,
                    a.rsu_lvl1,
                    a.rsu_lvl2
        FROM        df_with_rsu_flag a  
    ),
    df_lvl1_1_tuc AS (
        SELECT      a.material,
                    sum(lvl1_qty) as tuc_pc_m2
        FROM        df_lvl1_1_tuc_0 a
        WHERE       rsu_lvl1 = 'X'
        AND         rsu_lvl2 = 'X'
        GROUP BY    a.material
    ),
    df_lvl1_2_tuc AS (
        SELECT      a.material,
                    sum(lvl1_qty) as tuc_pc_m2
        FROM        df_lvl1_1_tuc_0 a
        WHERE       rsu_lvl1 = 'X'
        AND         rsu_lvl2 IS NULL
        AND         NOT EXISTS (SELECT 1 FROM df_lvl1_1_tuc b WHERE a.material = b.material)
        GROUP BY    a.material
    ),
    df_lvl2_tuc_0 AS (
        SELECT      a.material,
                    max(lvl1_qty) as lvl1_qty,
                    max(lvl2_qty) as lvl2_qty
        FROM        df_with_rsu_flag a
        WHERE       rsu_lvl1 IS NULL
        AND         rsu_lvl2 = 'X'
        GROUP BY    a.material
    ),
    df_lvl2_tuc AS (
        SELECT      material,
                    lvl1_qty * lvl2_qty as tuc_pc_m2
        FROM        df_lvl2_tuc_0
    ),
    df_final AS (
    SELECT * FROM df_lvl1_1_tuc
    UNION
    SELECT * FROM df_lvl1_2_tuc
    UNION
    SELECT * FROM df_lvl2_tuc
    )
    SELECT * FROM df_final
    """)
# display(df_tuc_product_chain)
df_tuc_product_chain.createOrReplaceTempView('df_tuc_product_chain')
df_tuc_product_chain.cache()
# record_count = df_tuc_product_chain.count()
# print(f"The total number of records in df_tuc_product_chain is: {record_count}")

df_product_portfolio_1 = spark.sql("""
WITH df_0 AS (
        SELECT      a.material_number,
                    coalesce(a.tuc_pc_m1,b.tuc_pc_m2) as tuc_pc
        FROM        df_tuc_marm a
        LEFT JOIN   df_tuc_product_chain b
        ON          a.material_number = b.material
    ),
    df_1 AS (
        SELECT      a.*,
                    coalesce (a.complex_tuc_qty,b.tuc_pc) as tuc_pc
        FROM        df_product_portfolio a
        LEFT JOIN   df_0 b
        ON          a.material_number = b.material_number
    )
    SELECT * FROM df_1 --where a.material_number = 'DJ54E'
""")
# display(df_product_portfolio_1)

print("2. CREATE TABLE")
create_table(metadata_json, df_product_portfolio_1)

In [29]:
%%sql 

SELECT m.Material, m.Traded_Unit, a.TDU_Traded_Unit, 
m.Retail_Sales_Unit, a.RSU_Retail_Sales_Unit, 
m.Merchandising_Unit, a.MCU_Merchandising_Unit, 
m.Intermediate_Product_Component, a.INT_Intermediate_Product_Component,
m.Semi_Finished_Product, a.SFP_Semi_Finished_Product

 FROM df_material m 
LEFT JOIN atlas_normalized.material_attributes a 
ON m.Material = a.Material_Number


